In [ ]:
import os
import logging
from openai import OpenAI
import google.generativeai as genai
from IPython.display import Markdown
import textwrap
from tqdm import tqdm
import tiktoken

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Set API keys from environment variables
# os.environ['OPENAI_API_KEY'] = '123'  # Replace with your actual API key
# os.environ['GOOGLE_API_KEY'] = '123'  # Replace with your actual Google API key
os.environ['OPENAI_API_KEY'] = ''  # Replace with your actual API key
os.environ['GOOGLE_API_KEY'] = ''  # Replace with your actual Google API key

# Check if API keys are set
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if not openai_api_key:
    logger.error("OpenAI API key is not set.")
if not google_api_key:
    logger.error("Google API key is not set.")

# Instantiate OpenAI client
openai_client = OpenAI(api_key=openai_api_key)

# Instantiate Google Generative AI client
genai.configure(api_key=google_api_key)

def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

def unified_api(question, model='gpt-4', context="You are a helpful assistant."):
    logger.info(f"Processing question with model {model}.")
    try:
        if model.startswith('gpt'):
            logger.debug("Using OpenAI model.")
            response = openai_client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "system", "content": context},
                    {"role": "user", "content": question}
                ]
            )
            result = response.choices[0].message.content.strip()
            logger.info("Received response from OpenAI.")
            return result
        
        elif model.startswith('gemini'):
            logger.debug("Using Google Generative AI model.")
            gemini_model = genai.GenerativeModel(model)
            response = gemini_model.generate_content(question)
            logger.info("Received response from Google Generative AI.")
            return response.text
            
        else:
            logger.error("Unsupported model type.")
            return "Unsupported model type."

    except Exception as e:
        logger.error(f"An error occurred: {e}")
        return None

# Shortcut functions for each model
def Q(question, context="You are a helpful assistant."):
    return unified_api(question, model='gpt-4o', context=context)

def Q3(question, context="You are a helpful assistant."):
    return unified_api(question, model='gpt-3.5-turbo', context=context)

def Q4(question, context="You are a helpful assistant."):
    return unified_api(question, model='gpt-4', context=context)

def G(question, context="You are a helpful assistant."):
    return unified_api(question, model='gemini-1.5-flash', context=context)
    
def GP(question, context="You are a helpful assistant."):
    return unified_api(question, model='gemini-1.5-pro', context=context)

def toksplit(text, max_tokens):
    logger.info("Splitting text into chunks.")
    tokenizer = tiktoken.encoding_for_model("gpt-4")
    tokens = tokenizer.encode(text)
    chunks = []
    current_chunk = []

    for token in tokens:
        current_chunk.append(token)
        if len(current_chunk) >= max_tokens:
            chunk_text = tokenizer.decode(current_chunk)
            chunks.append(chunk_text)
            current_chunk = []

    if current_chunk:
        chunk_text = tokenizer.decode(current_chunk)
        chunks.append(chunk_text)

    logger.info(f"Split text into {len(chunks)} chunks.")
    return chunks

def FILE(fname, query="?", chunk_size=6000, first=False, second=False, model_func=Q):
    logger.info(f"Processing file: {fname} with query: {query}")
    if not os.path.exists(fname):
        logger.error(f"File {fname} does not exist.")
        return "File does not exist."

    with open(fname, 'r', encoding='utf-8', errors='replace') as file:
        text = file.read()

    chunks = toksplit(text, chunk_size)
    results_accumulated = []

    for index, chunk in enumerate(tqdm(chunks, desc="Processing")):
        if first and index == 0:
            response = model_func(chunk + '\n\n' + query)
            results_accumulated.append(response)
            break
        elif second and index == 1:
            response = model_func(chunk + '\n\n' + query)
            results_accumulated.append(response)
            break
        elif not first and not second:
            response = model_func(chunk + '\n\n' + query)
            results_accumulated.append(response)

    all_responses = '\n'.join(results_accumulated)
    logger.info("File processing completed.")
    return all_responses

# Example usage
print(Q("What is the meaning of life?"))
print(Q3("What is the meaning of life?"))
print(Q4("What is the meaning of life?"))
print(G("What is the meaning of life?"))
print(GP("What is the meaning of life?"))

# File processing examples
# result = process_file('example.txt', "What are the key insights?", model_func=Q)
# print(result)


# ChatGPT4

In [36]:
STRATEGY=r"""\begin{table}[h]
\centering
\scriptsize
\caption{ChatGPT4 API output: Traffic Incident Mitigation Strategies}
\label{tab:traffic_incident_mitigation}
\begin{tabular}{|p{1cm}|p{2cm}|p{1cm}|p{3cm}|p{4cm}|p{4cm}|}
\hline
\textbf{Scenario ID} & \textbf{Incident Type} & \textbf{Severity} & \textbf{Location} & \textbf{Action} & \textbf{Equipment/Technology Required} \\
\hline
1 & Vehicle Breakdown & Low & Highway/Freeway Lane & - Deploy Incident Response Vehicle\newline - Temporary lane closure\newline - Use VMS to warn drivers & - Service Patrol Vehicle\newline - Variable Message Signs (VMS)\newline - Traffic Cones or Barriers \\
\hline
2 & Minor Two-Car Collision & Moderate & Urban Arterial & - Notify Police \& EMS if needed\newline - Quick clearance policy\newline - Use VMS \& social media & - Police/EMS Vehicles\newline - Quick Clearance Equipment (e.g., tow truck)\newline - Variable Message Signs\newline - Social Media Platforms \\
\hline
3 & Major Multi-Vehicle Crash & High & Highway/Freeway & - Full or partial lane closures\newline - Divert traffic to detour routes\newline - Activate EOC & - Police/EMS/Fire Services\newline - Emergency Operations Center (EOC) Activation\newline - Detour Signage \\
\hline
4 & Hazardous Material Spill & High & Near Urban Area & - Full road closure\newline - Mandatory evacuation if necessary\newline - HazMat Team dispatch & - HazMat Team\newline - Road Closure Signage\newline - Emergency Alert System (EAS) \\
\hline
5 & Overturned Truck & Moderate & Highway On-ramp/Off-ramp & - Partial ramp closure\newline - Speed limit reduction in area\newline - Deploy tow truck and cleanup crew & - Tow Trucks\newline - Cleanup Crew \newline - Speed Limit Signs \\
\hline
6 & Pedestrian Accident & High & Urban Crosswalk & - Full closure of affected lanes\newline - EMS priority dispatch\newline - Investigative procedures & - EMS Vehicles\newline - Police Investigation Unit\newline - Temporary Signage \\
\hline
7 & Wildlife on Road & Low & Rural Road & - Temporary speed limit reduction\newline - Use VMS to warn drivers\newline - Wildlife control dispatch & - Variable Message Signs\newline - Wildlife Control Services \\
\hline
8 & Infrastructure Failure (Bridge) & Very High & Bridge & - Full bridge closure\newline - Long-term detour setup\newline - Structural assessment & - Structural Engineering Team\newline - Permanent Detour Signage\newline - Media Briefing Equipment \\
\hline
9 & Snow/Ice Conditions & Variable & Major Roadways & - Speed limit reduction\newline - Deploy snow plows/salt trucks\newline - Use VMS and radio to inform & - Snow Plows/Salt Trucks\newline - Variable Message Signs\newline - Radio Broadcast System \\
\hline
10 & Fog/Visibility Issues & Variable & Highway/Freeway & - Speed limit reduction\newline - Flashing lights to warn drivers\newline - Use VMS to advise caution & - Variable Message Signs\newline - Highway Flashing Lights \\
\hline
\end{tabular}
\label{tab:guide}
\end{table}
"""

INCIDENTS = [
    "Accident ID A-4259643, Source Source1, Start Latitude 40.821863, Start Longitude -102.81153, End Latitude 40.815714, End Longitude -102.832031, Accident extent (miles) 1.153, Description Between US 6 (Sterling) and the Nebraska Border (4 miles east of Julesburg) from Mile Point 125 to Mile Point 184.14. Road closed expect delays due to a crash. Comment: Highway closed from Sterling to Nebraska state line due to adverse driving conditions and multiple crashes Between the Nebraska Border (4 miles east of Julesburg) and US 6 (Sterling) from Mile Point 184.14 to Mile Point 125. Road closed expect delays due to a crash. Comment: Highway closed from Sterling to Nebraska state line due to adverse driving conditions and multiple crashes, Street I-76, City Crook, County Logan, State CO, ZipCode 80726, Timezone US/Mountain, Airport Code KSNY, Temperature (F) 17.0, WindChill (F) -1.0, Humidity (%) 74.0, Pressure (inch) 25.51, Visibility (miles) 10.0, Wind Direction WNW, Wind Speed (mph) 25.0, Precipitation (inch) 0.0, Weather Condition Fair / Windy, Sunrise/Sunset Night, Civil Twilight Day, Nautical Twilight Day, Astronomical Twilight Day, Start_Time_hour 16, Start_Time_month 12, Weather_Timestamp_hour 16, Weather_Timestamp_month 12",
    "Accident ID A-5128843, Source Source1, Start Latitude 40.50956368427033, Start Longitude -103.80330204914186, End Latitude 40.49505368427033, End Longitude -103.80324204914186, Accident extent (miles) 1.003, Description Between County Road II and County Road 76 (6 miles west of the Raymer area) at Mile Point 104. Road closed expect delays due to a crash. Comment: CO52 closed both directions at MP104 about 7 miles south of CO14 due to crash alternate route advised. Between County Road 76 and County Road II (6 miles west of the Raymer area) at Mile Point 104. Road closed expect delays due to a crash. Comment: CO52 closed both directions at MP104 about 7 miles south of CO14 due to crash alternate route advised., Street  County Road JJ, City Fort Morgan, County Morgan, State CO, ZipCode 80701-9608, Timezone US/Mountain, Airport Code KFMM, Temperature (F) 34.0, WindChill (F) 27.0, Humidity (%) 70.0, Pressure (inch) 25.36, Visibility (miles) 10.0, Wind Direction WSW, Wind Speed (mph) 8.0, Precipitation (inch) 0.0, Weather Condition Fair, Stop, Sunrise/Sunset Day, Civil Twilight Day, Nautical Twilight Day, Astronomical Twilight Day, Start_Time_hour 8, Start_Time_month 3, Weather_Timestamp_hour 8, Weather_Timestamp_month 3",
    "Accident ID A-4227983, Source Source1, Start Latitude 37.762206, Start Longitude -104.835217, End Latitude 37.956072, End Longitude -104.800249, Accident extent (miles) 13.53, Description Between Exit 59: Butte Road (5 miles north of the Walsenburg area) and CO 165 (16 miles south of the Pueblo area) from Mile Point 60 to Mile Point 74. Road closed expect delays due to a crash. Slower speeds are advised. Comment: Seek alternate route use caution Between CO 165 (16 miles south of the Pueblo area) and Exit 59: Butte Road (5 miles north of the Walsenburg area) from Mile Point 74 to Mile Point 60. Road closed expect delays due to a crash. Slower speeds are advised. Comment: Seek alternate route use caution, Street I-25 N, City Walsenburg, County Huerfano, State CO, ZipCode 81089, Timezone US/Mountain, Airport Code KVTP, Temperature (F) 27.0, WindChill (F) 16.0, Humidity (%) 86.0, Pressure (inch) 20.32, Visibility (miles) 1.0, Wind Direction WSW, Wind Speed (mph) 13.0, Precipitation (inch) 0.0, Weather Condition Light Snow, Sunrise/Sunset Night, Civil Twilight Night, Nautical Twilight Night, Astronomical Twilight Night, Start_Time_hour 22, Start_Time_month 4, Weather_Timestamp_hour 22, Weather_Timestamp_month 4",
    "Accident ID A-4888575, Source Source1, Start Latitude 37.665370702563926, Start Longitude -102.62025475453079, End Latitude 37.70185070256392, End Longitude -102.60910475453079, Accident extent (miles) 2.593, Description Between County Road R (6 miles north of the Campo area) and Sage Street (near Lamar) from Mile Point 17 to Mile Point 74. Road closed expect delays due to a crash. Comment: Both directions closed expect delays. Watch for emergency crews. Between Sage Street (near Lamar) and County Road R (6 miles north of the Campo area) from Mile Point 74 to Mile Point 17. Road closed expect delays due to a crash. Comment: Both directions closed expect delays. Watch for emergency crews., Street US-287, City Lamar, County Prowers, State CO, ZipCode 81052, Timezone US/Mountain, Airport Code KLAA, Temperature (F) 51.0, WindChill (F) 51.0, Humidity (%) 80.0, Pressure (inch) 26.12, Visibility (miles) 10.0, Wind Direction E, Wind Speed (mph) 13.0, Precipitation (inch) 0.0, Weather Condition Cloudy, Sunrise/Sunset Day, Civil Twilight Day, Nautical Twilight Day, Astronomical Twilight Day, Start_Time_hour 12, Start_Time_month 5, Weather_Timestamp_hour 11, Weather_Timestamp_month 5",
    "Accident ID A-5968770, Source Source1, Start Latitude 37.2098886964931, Start Longitude -105.97833752581606, End Latitude 37.2255786964931, End Longitude -105.97419752581604, Accident extent (miles) 1.1079999999999999, Description Between County Road S and County Road T (1 mile north of the Romeo area) at Mile Point 16. Road closed expect delays due to a crash. Slower speeds are advised. Detour in place. Comment: Watch for detours onto County Road T5 Between County Road T and County Road S (1 mile north of the Romeo area) at Mile Point 16. Road closed expect delays due to a crash. Slower speeds are advised. Detour in place. Comment: Watch for detours onto County Road T5 , Street  County Road S, City La Jara, County Conejos, State CO, ZipCode 81140, Timezone US/Mountain, Airport Code KALS, Temperature (F) 55.0, WindChill (F) 55.0, Humidity (%) 24.0, Pressure (inch) 22.89, Visibility (miles) 10.0, Wind Direction CALM, Wind Speed (mph) 0.0, Precipitation (inch) 0.0, Weather Condition Fair, Sunrise/Sunset Day, Civil Twilight Day, Nautical Twilight Day, Astronomical Twilight Day, Start_Time_hour 15, Start_Time_month 10, Weather_Timestamp_hour 14, Weather_Timestamp_month 10",
    "Accident ID A-6137133, Source Source1, Start Latitude 40.954060999999996, Start Longitude -104.795343, End Latitude 40.950874, End Longitude -104.795581, Accident extent (miles) 0.221, Description Between County Road 128 and the Wyoming Border (13 miles north of the Nunn area) at Mile Point 306. Road closed expect delays due to a crash. Comment: US-85 closed in both directions at MP 306 4 miles South of Wyoming border.Between the Wyoming Border and County Road 128 (13 miles north of the Nunn area) at Mile Point 306. Road closed expect delays due to a crash. Comment: US-85 closed in both directions at MP 306 4 miles South of Wyoming border., Street  US Highway 85, City Carr, County Weld, State CO, ZipCode 80612-9201, Timezone US/Mountain, Airport Code KFEW, Temperature (F) 51.0, WindChill (F) 51.0, Humidity (%) 41.0, Pressure (inch) 23.88, Visibility (miles) 10.0, Wind Direction W, Wind Speed (mph) 16.0, Precipitation (inch) 0.0, Weather Condition Fair, Sunrise/Sunset Night, Civil Twilight Night, Nautical Twilight Night, Astronomical Twilight Night, Start_Time_hour 3, Start_Time_month 10, Weather_Timestamp_hour 2, Weather_Timestamp_month 10",
    "Accident ID A-4428281, Source Source1, Start Latitude 38.53234648686155, Start Longitude -104.96566414783017, End Latitude 38.47815648686155, End Longitude -104.97256414783016, Accident extent (miles) 3.763, Description Between County Road F45 and Barrett Road (10 miles north of the Florence area) at Mile Point 22. Road closed expect delays due to a crash. Comment: Watch for emergency responders. All lanes closed NB/SB. Expect delays. Between Barrett Road and County Road F45 (10 miles north of the Florence area) at Mile Point 22. Road closed expect delays due to a crash. Comment: Watch for emergency responders. All lanes closed NB/SB. Expect delays., Street  CO-115, City Penrose, County Fremont, State CO, ZipCode 81240, Timezone US/Mountain, Airport Code K1V6, Temperature (F) 63.0, WindChill (F) 63.0, Humidity (%) 27.0, Pressure (inch) 24.55, Visibility (miles) 10.0, Wind Direction SE, Wind Speed (mph) 6.0, Precipitation (inch) 0.0, Weather Condition Fair, Sunrise/Sunset Day, Civil Twilight Day, Nautical Twilight Day, Astronomical Twilight Day, Start_Time_hour 12, Start_Time_month 4, Weather_Timestamp_hour 12, Weather_Timestamp_month 4",
    "Accident ID A-4732415, Source Source1, Start Latitude 38.779571999999995, Start Longitude -107.950768, End Latitude 38.77187, End Longitude -107.96673600000001, Accident extent (miles) 1.011, Description Between 2100 Road and Lobdell Road (near Orchard City) from Mile Point 6 to Mile Point 7. Road closed expect delays due to a crash. Detour in place. Comment: Detour in place onto 875 turn left on 2200 road.Between Lobdell Road and 2100 Road (near Orchard City) from Mile Point 7 to Mile Point 6. Road closed expect delays due to a crash. Detour in place. Comment: Detour in place onto 875 turn left on 2200 road., Street  Highway 92, City Austin, County Delta, State CO, ZipCode 81410, Timezone US/Mountain, Airport Code KAJZ, Temperature (F) 19.0, WindChill (F) 11.0, Humidity (%) 77.0, Pressure (inch) 24.98, Visibility (miles) 10.0, Wind Direction N, Wind Speed (mph) 6.0, Precipitation (inch) 0.0, Weather Condition Partly Cloudy, Stop, Sunrise/Sunset Day, Civil Twilight Day, Nautical Twilight Day, Astronomical Twilight Day, Start_Time_hour 7, Start_Time_month 1, Weather_Timestamp_hour 7, Weather_Timestamp_month 1",
    "Accident ID A-6060568, Source Source1, Start Latitude 39.740563, Start Longitude -105.38991599999999, End Latitude 39.745005, End Longitude -105.39703200000001, Accident extent (miles) 0.48700000000000004, Description Between US 40; US 6 and CO 58; CO 93 (near Clear Creek Canyon) from Mile Point 260 to Mile Point 271. Road closed expect delays due to safety concerns. Comment: Due to crash. Use alternate route.Between CO 58; CO 93 and US 40; US 6 (near Clear Creek Canyon) from Mile Point 271 to Mile Point 260. Road closed expect delays due to safety concerns. Comment: Due to crash. Use alternate route., Street  US Highway 6, City Golden, County Jefferson, State CO, ZipCode 80403, Timezone US/Mountain, Airport Code KBDU, Temperature (F) 30.0, WindChill (F) 25.0, Humidity (%) 93.0, Pressure (inch) 24.35, Visibility (miles) 2.0, Wind Direction NE, Wind Speed (mph) 5.0, Precipitation (inch) 0.0, Weather Condition Light Snow, Sunrise/Sunset Day, Civil Twilight Day, Nautical Twilight Day, Astronomical Twilight Day, Start_Time_hour 8, Start_Time_month 12, Weather_Timestamp_hour 8, Weather_Timestamp_month 12",
    "Accident ID A-3996497, Source Source1, Start Latitude 36.285327, Start Longitude -86.271743, End Latitude 36.285496, End Longitude -86.271598, Accident extent (miles) 0.013999999999999999, Description State Route 10 northbound in Wilson County - Crash - Multiple Vehicles at PHILADELPHIA RD. reported at 8:12 PM 10/1/2022 and estimated to be cleared by 12:00 AM 10/2/2022 (Central Time Zone). Northbound traffic is affected with roadway closed. Southbound traffic is affected with delays expected. Northbound traffic is being diverted to alternate routes. Information provided by THP., Street  Hunters Point Pike, City Lebanon, County Wilson, State TN, ZipCode 37087-1129, Timezone US/Central, Airport Code KMQY, Temperature (F) 69.0, WindChill (F) 69.0, Humidity (%) 29.0, Pressure (inch) 29.38, Visibility (miles) 10.0, Wind Direction NNW, Wind Speed (mph) 7.0, Precipitation (inch) 0.0, Weather Condition Fair, Sunrise/Sunset Night, Civil Twilight Night, Nautical Twilight Night, Astronomical Twilight Night, Start_Time_hour 20, Start_Time_month 10, Weather_Timestamp_hour 19, Weather_Timestamp_month 10"
]

ACTIONS = r"""
Possible actions:
* Deploy IRV
* Temporary Lane Closure
* Use VMS to Warn Drivers
* Notify Police & EMS
* Quick Clearance Policy
* Use VMS & Social Media
* Full or Partial Lane Closures
* Divert Traffic to Detour Routes
* Activate EOC
* Full Road Closure
"""

PROMPT = r"""
Provide a binary array of actions to resolve the incident, based on ChatGPT4 API output: Traffic Incident Mitigation Strategies.
"""

RES =""
for INC in INCIDENTS:
    res = Q4(STRATEGY + INC + '\n\nIncident record: ' + ACTIONS + '\n\n'+PROMPT)
    print(res)
    RES+=res+"\n\n"

INFO:__main__:Processing question with model gpt-4.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Received response from OpenAI.
INFO:__main__:Processing question with model gpt-4.


Based on the incident details provided, the accident seems to be severe, occurring on a highway and involving multiple crashes due to adverse driving conditions. The location (I-76, a highway between Sterling and the Nebraska Border) and the incident type (crashes and adverse conditions) require strategic management. 

Assuming that the severity equates to "high" or "severe" and given the description of the accident, the likely actions to be taken from table are:

Scenario 3: Major Multi-Vehicle Crash
- Full or partial lane closures
- Divert traffic to detour routes
- Activate Emergency Operations Center (EOC)

Scenario 9: Snow/Ice Conditions (since Wind Chill is low, and Winds are high, assuming this is an Ice-related incident)
- Speed limit reduction
- Deploy snow plows/salt trucks
- Use VMS and radio to inform

Scenario 10: Fog/Visibility Issues (Visibility detail is optimal currently, but assuming it can reduce due to cold conditions)
- Speed limit reduction
- Flashing lights to wa

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Received response from OpenAI.
INFO:__main__:Processing question with model gpt-4.


Given the provided incident details, it appears to involve a crash on a highway, leading to a road closure with an advice for alternate routes. The severity is not directly provided, but due to the nature of the situation (full road closure), this can be inferred as a high severity event. Corresponding to Scenario ID 3 ("Major Multi-Vehicle Crash") and Severity 'High', the actions are as follows:

Deploy IRV - 0
Temporary Lane Closure - 0
Use VMS to Warn Drivers - 0
Notify Police & EMS - 0
Quick clearance policy - 0
Use VMS & Social Media - 0
Full or Partial Lane Closures - 1
Divert Traffic to Detour Routes - 1
Activate EOC - 1
Full road closure - 0

Therefore, the binary array of actions would be: [0, 0, 0, 0, 0, 0, 1, 1, 1, 0]
Note: The binary representation is based on whether the incident mitigation strategy is used or not (1 if used, 0 if not used).


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Received response from OpenAI.
INFO:__main__:Processing question with model gpt-4.


...

Based on the information provided about the accident and referring to the table of Traffic Incident Mitigation Strategies, the most appropriate actions seem to be:

1. Full or partial lane closures- Due to Severity of incident and Long extent in Miles
2. Divert traffic to detour routes - As described in the Accident description, a detour is already in place
3. Notify Police & EMS - As this is a crash, emergency services are likely required

Therefore, according to the table index:

1 (Deploy Incident Response Vehicle): 0
2 (Temporary lane closure): 0
3 (Use VMS to warn drivers): 0
4 (Notify Police & EMS if needed): 1
5 (Quick clearance policy): 0
6 (Use VMS & social media): 0
7 (Full or partial lane closures): 1
8 (Divert traffic to detour routes): 1
9 (Activate EOC): 0
10 (Full road closure): 0

The binary array would be: [0, 0, 0, 1, 0, 0, 1, 1, 0, 0]


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Received response from OpenAI.
INFO:__main__:Processing question with model gpt-4.


Based on the provided incident description, it appears that there was a crash on the road between Mile Point 260 to Mile Point 271 near Clear Creek Canyon, causing roads to be closed. Given the severity of the incident, the actions needed to manage the situation can be mapped from Table \ref{tab:traffic_incident_mitigation}. Here is the binary array mapping the actions given in the incident table: 

1. Deploy Incident Response Vehicle: Yes (1)
2. Temporary lane closure: Yes (1)
3. Use VMS to warn drivers: Yes (1)
4. Notify Police & EMS if needed: Yes (1)
5. Quick clearance policy: Yes (1)
6. Use VMS & social media: Yes (1)
7. Full or partial lane closures: Yes (1)
8. Divert traffic to detour routes: Yes (1)
9. Activate EOC: Yes (1)
10. Full road closure: Yes (1)
11. Mandatory evacuation: No (0)
12. HazMat Team dispatch: No (0)
13. Partial ramp closure: No (0)
14. Speed limit reduction in area: No (0)
15. Deploy tow truck and cleanup crew: No (0)
16. Full closure of affected lanes: No (

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Received response from OpenAI.


Given the incident description which seems to indicate a multiple vehicle crash and the severity is not explicitly stated, we could infer a high degree of severity due to the fact that the road is closed and traffic is being diverted. 

Therefore, we apply a binary array of actions from the 'Major Multi-Vehicle Crash' scenario (Scenario ID 3) from the table:

[0, 0, 1, 1, 1, 0]

This represents:

- Deploy Incident Response Vehicle: No (0)
- Temporary Lane Closure: No (0)
- Use VMS to Warn Drivers: Yes (1)
- Notify Police & EMS: Yes (1)
- Quick Clearance Policy: Yes (1)
- Use VMS & Social Media: No (0) 

Please adapt the binary array if you think that the situation fits better another scenario from the table.


# Gemini

In [41]:
STRATEGY=r"""\begin{table}[h]
\centering
\scriptsize
\caption{ChatGPT4 API output: Traffic Incident Mitigation Strategies}
\label{tab:traffic_incident_mitigation}
\begin{tabular}{|p{1cm}|p{2cm}|p{1cm}|p{3cm}|p{4cm}|p{4cm}|}
\hline
\textbf{Scenario ID} & \textbf{Incident Type} & \textbf{Severity} & \textbf{Location} & \textbf{Action} & \textbf{Equipment/Technology Required} \\
\hline
1 & Vehicle Breakdown & Low & Highway/Freeway Lane & - Deploy Incident Response Vehicle\newline - Temporary lane closure\newline - Use VMS to warn drivers & - Service Patrol Vehicle\newline - Variable Message Signs (VMS)\newline - Traffic Cones or Barriers \\
\hline
2 & Minor Two-Car Collision & Moderate & Urban Arterial & - Notify Police \& EMS if needed\newline - Quick clearance policy\newline - Use VMS \& social media & - Police/EMS Vehicles\newline - Quick Clearance Equipment (e.g., tow truck)\newline - Variable Message Signs\newline - Social Media Platforms \\
\hline
3 & Major Multi-Vehicle Crash & High & Highway/Freeway & - Full or partial lane closures\newline - Divert traffic to detour routes\newline - Activate EOC & - Police/EMS/Fire Services\newline - Emergency Operations Center (EOC) Activation\newline - Detour Signage \\
\hline
4 & Hazardous Material Spill & High & Near Urban Area & - Full road closure\newline - Mandatory evacuation if necessary\newline - HazMat Team dispatch & - HazMat Team\newline - Road Closure Signage\newline - Emergency Alert System (EAS) \\
\hline
5 & Overturned Truck & Moderate & Highway On-ramp/Off-ramp & - Partial ramp closure\newline - Speed limit reduction in area\newline - Deploy tow truck and cleanup crew & - Tow Trucks\newline - Cleanup Crew \newline - Speed Limit Signs \\
\hline
6 & Pedestrian Accident & High & Urban Crosswalk & - Full closure of affected lanes\newline - EMS priority dispatch\newline - Investigative procedures & - EMS Vehicles\newline - Police Investigation Unit\newline - Temporary Signage \\
\hline
7 & Wildlife on Road & Low & Rural Road & - Temporary speed limit reduction\newline - Use VMS to warn drivers\newline - Wildlife control dispatch & - Variable Message Signs\newline - Wildlife Control Services \\
\hline
8 & Infrastructure Failure (Bridge) & Very High & Bridge & - Full bridge closure\newline - Long-term detour setup\newline - Structural assessment & - Structural Engineering Team\newline - Permanent Detour Signage\newline - Media Briefing Equipment \\
\hline
9 & Snow/Ice Conditions & Variable & Major Roadways & - Speed limit reduction\newline - Deploy snow plows/salt trucks\newline - Use VMS and radio to inform & - Snow Plows/Salt Trucks\newline - Variable Message Signs\newline - Radio Broadcast System \\
\hline
10 & Fog/Visibility Issues & Variable & Highway/Freeway & - Speed limit reduction\newline - Flashing lights to warn drivers\newline - Use VMS to advise caution & - Variable Message Signs\newline - Highway Flashing Lights \\
\hline
\end{tabular}
\label{tab:guide}
\end{table}
"""

INCIDENTS = [
    "Accident ID A-4259643, Source Source1, Start Latitude 40.821863, Start Longitude -102.81153, End Latitude 40.815714, End Longitude -102.832031, Accident extent (miles) 1.153, Description Between US 6 (Sterling) and the Nebraska Border (4 miles east of Julesburg) from Mile Point 125 to Mile Point 184.14. Road closed expect delays due to a crash. Comment: Highway closed from Sterling to Nebraska state line due to adverse driving conditions and multiple crashes Between the Nebraska Border (4 miles east of Julesburg) and US 6 (Sterling) from Mile Point 184.14 to Mile Point 125. Road closed expect delays due to a crash. Comment: Highway closed from Sterling to Nebraska state line due to adverse driving conditions and multiple crashes, Street I-76, City Crook, County Logan, State CO, ZipCode 80726, Timezone US/Mountain, Airport Code KSNY, Temperature (F) 17.0, WindChill (F) -1.0, Humidity (%) 74.0, Pressure (inch) 25.51, Visibility (miles) 10.0, Wind Direction WNW, Wind Speed (mph) 25.0, Precipitation (inch) 0.0, Weather Condition Fair / Windy, Sunrise/Sunset Night, Civil Twilight Day, Nautical Twilight Day, Astronomical Twilight Day, Start_Time_hour 16, Start_Time_month 12, Weather_Timestamp_hour 16, Weather_Timestamp_month 12",
    "Accident ID A-5128843, Source Source1, Start Latitude 40.50956368427033, Start Longitude -103.80330204914186, End Latitude 40.49505368427033, End Longitude -103.80324204914186, Accident extent (miles) 1.003, Description Between County Road II and County Road 76 (6 miles west of the Raymer area) at Mile Point 104. Road closed expect delays due to a crash. Comment: CO52 closed both directions at MP104 about 7 miles south of CO14 due to crash alternate route advised. Between County Road 76 and County Road II (6 miles west of the Raymer area) at Mile Point 104. Road closed expect delays due to a crash. Comment: CO52 closed both directions at MP104 about 7 miles south of CO14 due to crash alternate route advised., Street  County Road JJ, City Fort Morgan, County Morgan, State CO, ZipCode 80701-9608, Timezone US/Mountain, Airport Code KFMM, Temperature (F) 34.0, WindChill (F) 27.0, Humidity (%) 70.0, Pressure (inch) 25.36, Visibility (miles) 10.0, Wind Direction WSW, Wind Speed (mph) 8.0, Precipitation (inch) 0.0, Weather Condition Fair, Stop, Sunrise/Sunset Day, Civil Twilight Day, Nautical Twilight Day, Astronomical Twilight Day, Start_Time_hour 8, Start_Time_month 3, Weather_Timestamp_hour 8, Weather_Timestamp_month 3",
    "Accident ID A-4227983, Source Source1, Start Latitude 37.762206, Start Longitude -104.835217, End Latitude 37.956072, End Longitude -104.800249, Accident extent (miles) 13.53, Description Between Exit 59: Butte Road (5 miles north of the Walsenburg area) and CO 165 (16 miles south of the Pueblo area) from Mile Point 60 to Mile Point 74. Road closed expect delays due to a crash. Slower speeds are advised. Comment: Seek alternate route use caution Between CO 165 (16 miles south of the Pueblo area) and Exit 59: Butte Road (5 miles north of the Walsenburg area) from Mile Point 74 to Mile Point 60. Road closed expect delays due to a crash. Slower speeds are advised. Comment: Seek alternate route use caution, Street I-25 N, City Walsenburg, County Huerfano, State CO, ZipCode 81089, Timezone US/Mountain, Airport Code KVTP, Temperature (F) 27.0, WindChill (F) 16.0, Humidity (%) 86.0, Pressure (inch) 20.32, Visibility (miles) 1.0, Wind Direction WSW, Wind Speed (mph) 13.0, Precipitation (inch) 0.0, Weather Condition Light Snow, Sunrise/Sunset Night, Civil Twilight Night, Nautical Twilight Night, Astronomical Twilight Night, Start_Time_hour 22, Start_Time_month 4, Weather_Timestamp_hour 22, Weather_Timestamp_month 4",
    "Accident ID A-4888575, Source Source1, Start Latitude 37.665370702563926, Start Longitude -102.62025475453079, End Latitude 37.70185070256392, End Longitude -102.60910475453079, Accident extent (miles) 2.593, Description Between County Road R (6 miles north of the Campo area) and Sage Street (near Lamar) from Mile Point 17 to Mile Point 74. Road closed expect delays due to a crash. Comment: Both directions closed expect delays. Watch for emergency crews. Between Sage Street (near Lamar) and County Road R (6 miles north of the Campo area) from Mile Point 74 to Mile Point 17. Road closed expect delays due to a crash. Comment: Both directions closed expect delays. Watch for emergency crews., Street US-287, City Lamar, County Prowers, State CO, ZipCode 81052, Timezone US/Mountain, Airport Code KLAA, Temperature (F) 51.0, WindChill (F) 51.0, Humidity (%) 80.0, Pressure (inch) 26.12, Visibility (miles) 10.0, Wind Direction E, Wind Speed (mph) 13.0, Precipitation (inch) 0.0, Weather Condition Cloudy, Sunrise/Sunset Day, Civil Twilight Day, Nautical Twilight Day, Astronomical Twilight Day, Start_Time_hour 12, Start_Time_month 5, Weather_Timestamp_hour 11, Weather_Timestamp_month 5",
    "Accident ID A-5968770, Source Source1, Start Latitude 37.2098886964931, Start Longitude -105.97833752581606, End Latitude 37.2255786964931, End Longitude -105.97419752581604, Accident extent (miles) 1.1079999999999999, Description Between County Road S and County Road T (1 mile north of the Romeo area) at Mile Point 16. Road closed expect delays due to a crash. Slower speeds are advised. Detour in place. Comment: Watch for detours onto County Road T5 Between County Road T and County Road S (1 mile north of the Romeo area) at Mile Point 16. Road closed expect delays due to a crash. Slower speeds are advised. Detour in place. Comment: Watch for detours onto County Road T5 , Street  County Road S, City La Jara, County Conejos, State CO, ZipCode 81140, Timezone US/Mountain, Airport Code KALS, Temperature (F) 55.0, WindChill (F) 55.0, Humidity (%) 24.0, Pressure (inch) 22.89, Visibility (miles) 10.0, Wind Direction CALM, Wind Speed (mph) 0.0, Precipitation (inch) 0.0, Weather Condition Fair, Sunrise/Sunset Day, Civil Twilight Day, Nautical Twilight Day, Astronomical Twilight Day, Start_Time_hour 15, Start_Time_month 10, Weather_Timestamp_hour 14, Weather_Timestamp_month 10",
    "Accident ID A-6137133, Source Source1, Start Latitude 40.954060999999996, Start Longitude -104.795343, End Latitude 40.950874, End Longitude -104.795581, Accident extent (miles) 0.221, Description Between County Road 128 and the Wyoming Border (13 miles north of the Nunn area) at Mile Point 306. Road closed expect delays due to a crash. Comment: US-85 closed in both directions at MP 306 4 miles South of Wyoming border.Between the Wyoming Border and County Road 128 (13 miles north of the Nunn area) at Mile Point 306. Road closed expect delays due to a crash. Comment: US-85 closed in both directions at MP 306 4 miles South of Wyoming border., Street  US Highway 85, City Carr, County Weld, State CO, ZipCode 80612-9201, Timezone US/Mountain, Airport Code KFEW, Temperature (F) 51.0, WindChill (F) 51.0, Humidity (%) 41.0, Pressure (inch) 23.88, Visibility (miles) 10.0, Wind Direction W, Wind Speed (mph) 16.0, Precipitation (inch) 0.0, Weather Condition Fair, Sunrise/Sunset Night, Civil Twilight Night, Nautical Twilight Night, Astronomical Twilight Night, Start_Time_hour 3, Start_Time_month 10, Weather_Timestamp_hour 2, Weather_Timestamp_month 10",
    "Accident ID A-4428281, Source Source1, Start Latitude 38.53234648686155, Start Longitude -104.96566414783017, End Latitude 38.47815648686155, End Longitude -104.97256414783016, Accident extent (miles) 3.763, Description Between County Road F45 and Barrett Road (10 miles north of the Florence area) at Mile Point 22. Road closed expect delays due to a crash. Comment: Watch for emergency responders. All lanes closed NB/SB. Expect delays. Between Barrett Road and County Road F45 (10 miles north of the Florence area) at Mile Point 22. Road closed expect delays due to a crash. Comment: Watch for emergency responders. All lanes closed NB/SB. Expect delays., Street  CO-115, City Penrose, County Fremont, State CO, ZipCode 81240, Timezone US/Mountain, Airport Code K1V6, Temperature (F) 63.0, WindChill (F) 63.0, Humidity (%) 27.0, Pressure (inch) 24.55, Visibility (miles) 10.0, Wind Direction SE, Wind Speed (mph) 6.0, Precipitation (inch) 0.0, Weather Condition Fair, Sunrise/Sunset Day, Civil Twilight Day, Nautical Twilight Day, Astronomical Twilight Day, Start_Time_hour 12, Start_Time_month 4, Weather_Timestamp_hour 12, Weather_Timestamp_month 4",
    "Accident ID A-4732415, Source Source1, Start Latitude 38.779571999999995, Start Longitude -107.950768, End Latitude 38.77187, End Longitude -107.96673600000001, Accident extent (miles) 1.011, Description Between 2100 Road and Lobdell Road (near Orchard City) from Mile Point 6 to Mile Point 7. Road closed expect delays due to a crash. Detour in place. Comment: Detour in place onto 875 turn left on 2200 road.Between Lobdell Road and 2100 Road (near Orchard City) from Mile Point 7 to Mile Point 6. Road closed expect delays due to a crash. Detour in place. Comment: Detour in place onto 875 turn left on 2200 road., Street  Highway 92, City Austin, County Delta, State CO, ZipCode 81410, Timezone US/Mountain, Airport Code KAJZ, Temperature (F) 19.0, WindChill (F) 11.0, Humidity (%) 77.0, Pressure (inch) 24.98, Visibility (miles) 10.0, Wind Direction N, Wind Speed (mph) 6.0, Precipitation (inch) 0.0, Weather Condition Partly Cloudy, Stop, Sunrise/Sunset Day, Civil Twilight Day, Nautical Twilight Day, Astronomical Twilight Day, Start_Time_hour 7, Start_Time_month 1, Weather_Timestamp_hour 7, Weather_Timestamp_month 1",
    "Accident ID A-6060568, Source Source1, Start Latitude 39.740563, Start Longitude -105.38991599999999, End Latitude 39.745005, End Longitude -105.39703200000001, Accident extent (miles) 0.48700000000000004, Description Between US 40; US 6 and CO 58; CO 93 (near Clear Creek Canyon) from Mile Point 260 to Mile Point 271. Road closed expect delays due to safety concerns. Comment: Due to crash. Use alternate route.Between CO 58; CO 93 and US 40; US 6 (near Clear Creek Canyon) from Mile Point 271 to Mile Point 260. Road closed expect delays due to safety concerns. Comment: Due to crash. Use alternate route., Street  US Highway 6, City Golden, County Jefferson, State CO, ZipCode 80403, Timezone US/Mountain, Airport Code KBDU, Temperature (F) 30.0, WindChill (F) 25.0, Humidity (%) 93.0, Pressure (inch) 24.35, Visibility (miles) 2.0, Wind Direction NE, Wind Speed (mph) 5.0, Precipitation (inch) 0.0, Weather Condition Light Snow, Sunrise/Sunset Day, Civil Twilight Day, Nautical Twilight Day, Astronomical Twilight Day, Start_Time_hour 8, Start_Time_month 12, Weather_Timestamp_hour 8, Weather_Timestamp_month 12",
    "Accident ID A-3996497, Source Source1, Start Latitude 36.285327, Start Longitude -86.271743, End Latitude 36.285496, End Longitude -86.271598, Accident extent (miles) 0.013999999999999999, Description State Route 10 northbound in Wilson County - Crash - Multiple Vehicles at PHILADELPHIA RD. reported at 8:12 PM 10/1/2022 and estimated to be cleared by 12:00 AM 10/2/2022 (Central Time Zone). Northbound traffic is affected with roadway closed. Southbound traffic is affected with delays expected. Northbound traffic is being diverted to alternate routes. Information provided by THP., Street  Hunters Point Pike, City Lebanon, County Wilson, State TN, ZipCode 37087-1129, Timezone US/Central, Airport Code KMQY, Temperature (F) 69.0, WindChill (F) 69.0, Humidity (%) 29.0, Pressure (inch) 29.38, Visibility (miles) 10.0, Wind Direction NNW, Wind Speed (mph) 7.0, Precipitation (inch) 0.0, Weather Condition Fair, Sunrise/Sunset Night, Civil Twilight Night, Nautical Twilight Night, Astronomical Twilight Night, Start_Time_hour 20, Start_Time_month 10, Weather_Timestamp_hour 19, Weather_Timestamp_month 10"
]

ACTIONS = r"""
Possible actions:
* Deploy IRV
* Temporary Lane Closure
* Use VMS to Warn Drivers
* Notify Police & EMS
* Quick Clearance Policy
* Use VMS & Social Media
* Full or Partial Lane Closures
* Divert Traffic to Detour Routes
* Activate EOC
* Full Road Closure
"""

PROMPT = r"""
Provide a binary array of actions to resolve the incident, based on ChatGPT4 API output: Traffic Incident Mitigation Strategies.
"""

RES =""
for INC in INCIDENTS:
    res = G(STRATEGY + INC + '\n\nIncident record: ' + ACTIONS + '\n\n'+PROMPT)
    print(res)
    RES+=INC+":\n\n"+res+"\n\n"

INFO:__main__:Processing question with model gemini-1.5-flash.
INFO:__main__:Received response from Google Generative AI.
INFO:__main__:Processing question with model gemini-1.5-flash.


Here's the binary array of actions based on the provided incident description and the ChatGPT4 API output table:

**Incident Description:**

* **Location:** I-76, between US 6 (Sterling) and the Nebraska Border (4 miles east of Julesburg) 
* **Road Closure:** Highway closed from Sterling to the Nebraska state line
* **Cause:** Adverse driving conditions and multiple crashes
* **Weather:** Fair/Windy, Visibility 10 miles, Wind Speed 25 mph 

**Analysis:**

This incident seems to involve multiple crashes due to weather conditions, requiring a full road closure and comprehensive mitigation measures.

**Binary Action Array:**

* **Deploy IRV:**  **1**  (Likely needed for multiple crashes)
* **Temporary Lane Closure:** **0** (Full closure in effect)
* **Use VMS to Warn Drivers:** **1** (Essential for informing drivers of the closure)
* **Notify Police & EMS:** **1** (Multiple crashes likely require emergency services)
* **Quick Clearance Policy:** **1** (Necessary for efficient removal of v

INFO:__main__:Received response from Google Generative AI.
INFO:__main__:Processing question with model gemini-1.5-flash.


Here's the breakdown of the actions based on the provided incident description and the ChatGPT4 API output:

**Incident Analysis:**

* **Incident Type:** Crash (implied by "Road closed expect delays due to a crash")
* **Severity:** Moderate (Likely, given "Road closed" but not a multi-vehicle collision)
* **Location:** Highway/Freeway (CO52)
* **Additional Information:**  Closed in both directions, detour advised.

**Action Array (Binary - 1 for yes, 0 for no):**

* **Deploy IRV (Incident Response Vehicle):** 1 (Recommended for any incident on a highway)
* **Temporary Lane Closure:** 1 (Essential given road closure)
* **Use VMS to Warn Drivers:** 1 (Crucial for informing drivers about the incident)
* **Notify Police & EMS:** 1 (Likely needed due to a crash, even if no injuries reported) 
* **Quick Clearance Policy:** 1 (Appropriate for a likely moderate crash to minimize delays)
* **Use VMS & Social Media:** 1 (To provide information to the public)
* **Full or Partial Lane Closures:** 

INFO:__main__:Received response from Google Generative AI.
INFO:__main__:Processing question with model gemini-1.5-flash.


...

Based on the provided information and the table of Traffic Incident Mitigation Strategies, here's a binary array representing the actions to resolve the incident:

```
[1, 1, 1, 1, 1, 1, 0, 0, 0, 0]
```

Here's the breakdown:

* **[1, 1, 1]** - **Deploy IRV, Temporary Lane Closure, Use VMS to Warn Drivers**: These are standard actions for a traffic incident, especially when a road closure is in place.
* **[1, 1]** - **Notify Police & EMS, Quick Clearance Policy**:  Given the crash and road closure, police and EMS should be notified, and a quick clearance policy is essential. 
* **[1]** - **Use VMS & Social Media**:  VMS is already mentioned, and social media is a valuable tool for disseminating information about closures and detours. 
* **[0, 0, 0, 0]** - **Full or Partial Lane Closures, Divert Traffic to Detour Routes, Activate EOC, Full Road Closure**: While the road is closed, the description already mentions a detour. Full closure is implied by the "Road Closed" message. Activating

INFO:__main__:Received response from Google Generative AI.
INFO:__main__:Processing question with model gemini-1.5-flash.


Here's the breakdown of actions and the binary array:

**Incident Analysis:**

* **Incident Type:**  Crash (implied from "Road closed due to safety concerns.")
* **Severity:** Moderate to High (Road closure indicates a significant event)
* **Location:** Highway (US Highway 6)
* **Additional Factors:**  Snow (Weather Condition: Light Snow), Visibility (2.0 miles)

**Possible Actions (from the provided text):**

* **Deploy IRV (Incident Response Vehicle):**  Yes, highly likely for initial assessment and traffic control.
* **Temporary Lane Closure:** Yes, necessary for safety and crash investigation. 
* **Use VMS to Warn Drivers:**  Yes, essential to alert approaching traffic.
* **Notify Police & EMS:** Yes, due to the crash and potential injuries.
* **Quick Clearance Policy:**  Potentially, but likely not immediately feasible given the snow and visibility.
* **Use VMS & Social Media:**  Yes, for broader traffic advisories and route guidance.
* **Full or Partial Lane Closures:** Yes, give

INFO:__main__:Received response from Google Generative AI.


Here's the binary array of actions based on the incident description and the ChatGPT4 table:

```
[1, 1, 1, 1, 0, 1, 1, 0, 0, 0] 
```

Here's the breakdown of why each action is included or excluded:

* **1. Deploy IRV:**  **Yes (1)**  -  A vehicle breakdown is a common reason to deploy an Incident Response Vehicle (IRV).
* **2. Temporary Lane Closure:** **Yes (1)** - The description mentions a roadway closure, indicating a need for temporary lane closure.
* **3. Use VMS to Warn Drivers:** **Yes (1)** -  The description mentions using VMS (Variable Message Signs), and VMS is a standard practice for traffic incidents.
* **4. Notify Police & EMS:** **Yes (1)** -  Multiple vehicles were involved in the crash, making it likely that police and EMS would be needed.
* **5. Quick Clearance Policy:** **No (0)** -  The description suggests a longer-term closure, not a quick clearance situation.
* **6. Use VMS & Social Media:** **Yes (1)** - The description mentions using VMS and the incident is 

# ChatGPT 3.5 - Turbo

In [45]:
STRATEGY=r"""\begin{table}[h]
\centering
\scriptsize
\caption{ChatGPT4 API output: Traffic Incident Mitigation Strategies}
\label{tab:traffic_incident_mitigation}
\begin{tabular}{|p{1cm}|p{2cm}|p{1cm}|p{3cm}|p{4cm}|p{4cm}|}
\hline
\textbf{Scenario ID} & \textbf{Incident Type} & \textbf{Severity} & \textbf{Location} & \textbf{Action} & \textbf{Equipment/Technology Required} \\
\hline
1 & Vehicle Breakdown & Low & Highway/Freeway Lane & - Deploy Incident Response Vehicle\newline - Temporary lane closure\newline - Use VMS to warn drivers & - Service Patrol Vehicle\newline - Variable Message Signs (VMS)\newline - Traffic Cones or Barriers \\
\hline
2 & Minor Two-Car Collision & Moderate & Urban Arterial & - Notify Police \& EMS if needed\newline - Quick clearance policy\newline - Use VMS \& social media & - Police/EMS Vehicles\newline - Quick Clearance Equipment (e.g., tow truck)\newline - Variable Message Signs\newline - Social Media Platforms \\
\hline
3 & Major Multi-Vehicle Crash & High & Highway/Freeway & - Full or partial lane closures\newline - Divert traffic to detour routes\newline - Activate EOC & - Police/EMS/Fire Services\newline - Emergency Operations Center (EOC) Activation\newline - Detour Signage \\
\hline
4 & Hazardous Material Spill & High & Near Urban Area & - Full road closure\newline - Mandatory evacuation if necessary\newline - HazMat Team dispatch & - HazMat Team\newline - Road Closure Signage\newline - Emergency Alert System (EAS) \\
\hline
5 & Overturned Truck & Moderate & Highway On-ramp/Off-ramp & - Partial ramp closure\newline - Speed limit reduction in area\newline - Deploy tow truck and cleanup crew & - Tow Trucks\newline - Cleanup Crew \newline - Speed Limit Signs \\
\hline
6 & Pedestrian Accident & High & Urban Crosswalk & - Full closure of affected lanes\newline - EMS priority dispatch\newline - Investigative procedures & - EMS Vehicles\newline - Police Investigation Unit\newline - Temporary Signage \\
\hline
7 & Wildlife on Road & Low & Rural Road & - Temporary speed limit reduction\newline - Use VMS to warn drivers\newline - Wildlife control dispatch & - Variable Message Signs\newline - Wildlife Control Services \\
\hline
8 & Infrastructure Failure (Bridge) & Very High & Bridge & - Full bridge closure\newline - Long-term detour setup\newline - Structural assessment & - Structural Engineering Team\newline - Permanent Detour Signage\newline - Media Briefing Equipment \\
\hline
9 & Snow/Ice Conditions & Variable & Major Roadways & - Speed limit reduction\newline - Deploy snow plows/salt trucks\newline - Use VMS and radio to inform & - Snow Plows/Salt Trucks\newline - Variable Message Signs\newline - Radio Broadcast System \\
\hline
10 & Fog/Visibility Issues & Variable & Highway/Freeway & - Speed limit reduction\newline - Flashing lights to warn drivers\newline - Use VMS to advise caution & - Variable Message Signs\newline - Highway Flashing Lights \\
\hline
\end{tabular}
\label{tab:guide}
\end{table}
"""

INCIDENTS = [
    "Accident ID A-4259643, Source Source1, Start Latitude 40.821863, Start Longitude -102.81153, End Latitude 40.815714, End Longitude -102.832031, Accident extent (miles) 1.153, Description Between US 6 (Sterling) and the Nebraska Border (4 miles east of Julesburg) from Mile Point 125 to Mile Point 184.14. Road closed expect delays due to a crash. Comment: Highway closed from Sterling to Nebraska state line due to adverse driving conditions and multiple crashes Between the Nebraska Border (4 miles east of Julesburg) and US 6 (Sterling) from Mile Point 184.14 to Mile Point 125. Road closed expect delays due to a crash. Comment: Highway closed from Sterling to Nebraska state line due to adverse driving conditions and multiple crashes, Street I-76, City Crook, County Logan, State CO, ZipCode 80726, Timezone US/Mountain, Airport Code KSNY, Temperature (F) 17.0, WindChill (F) -1.0, Humidity (%) 74.0, Pressure (inch) 25.51, Visibility (miles) 10.0, Wind Direction WNW, Wind Speed (mph) 25.0, Precipitation (inch) 0.0, Weather Condition Fair / Windy, Sunrise/Sunset Night, Civil Twilight Day, Nautical Twilight Day, Astronomical Twilight Day, Start_Time_hour 16, Start_Time_month 12, Weather_Timestamp_hour 16, Weather_Timestamp_month 12",
    "Accident ID A-5128843, Source Source1, Start Latitude 40.50956368427033, Start Longitude -103.80330204914186, End Latitude 40.49505368427033, End Longitude -103.80324204914186, Accident extent (miles) 1.003, Description Between County Road II and County Road 76 (6 miles west of the Raymer area) at Mile Point 104. Road closed expect delays due to a crash. Comment: CO52 closed both directions at MP104 about 7 miles south of CO14 due to crash alternate route advised. Between County Road 76 and County Road II (6 miles west of the Raymer area) at Mile Point 104. Road closed expect delays due to a crash. Comment: CO52 closed both directions at MP104 about 7 miles south of CO14 due to crash alternate route advised., Street  County Road JJ, City Fort Morgan, County Morgan, State CO, ZipCode 80701-9608, Timezone US/Mountain, Airport Code KFMM, Temperature (F) 34.0, WindChill (F) 27.0, Humidity (%) 70.0, Pressure (inch) 25.36, Visibility (miles) 10.0, Wind Direction WSW, Wind Speed (mph) 8.0, Precipitation (inch) 0.0, Weather Condition Fair, Stop, Sunrise/Sunset Day, Civil Twilight Day, Nautical Twilight Day, Astronomical Twilight Day, Start_Time_hour 8, Start_Time_month 3, Weather_Timestamp_hour 8, Weather_Timestamp_month 3",
    "Accident ID A-4227983, Source Source1, Start Latitude 37.762206, Start Longitude -104.835217, End Latitude 37.956072, End Longitude -104.800249, Accident extent (miles) 13.53, Description Between Exit 59: Butte Road (5 miles north of the Walsenburg area) and CO 165 (16 miles south of the Pueblo area) from Mile Point 60 to Mile Point 74. Road closed expect delays due to a crash. Slower speeds are advised. Comment: Seek alternate route use caution Between CO 165 (16 miles south of the Pueblo area) and Exit 59: Butte Road (5 miles north of the Walsenburg area) from Mile Point 74 to Mile Point 60. Road closed expect delays due to a crash. Slower speeds are advised. Comment: Seek alternate route use caution, Street I-25 N, City Walsenburg, County Huerfano, State CO, ZipCode 81089, Timezone US/Mountain, Airport Code KVTP, Temperature (F) 27.0, WindChill (F) 16.0, Humidity (%) 86.0, Pressure (inch) 20.32, Visibility (miles) 1.0, Wind Direction WSW, Wind Speed (mph) 13.0, Precipitation (inch) 0.0, Weather Condition Light Snow, Sunrise/Sunset Night, Civil Twilight Night, Nautical Twilight Night, Astronomical Twilight Night, Start_Time_hour 22, Start_Time_month 4, Weather_Timestamp_hour 22, Weather_Timestamp_month 4",
    "Accident ID A-4888575, Source Source1, Start Latitude 37.665370702563926, Start Longitude -102.62025475453079, End Latitude 37.70185070256392, End Longitude -102.60910475453079, Accident extent (miles) 2.593, Description Between County Road R (6 miles north of the Campo area) and Sage Street (near Lamar) from Mile Point 17 to Mile Point 74. Road closed expect delays due to a crash. Comment: Both directions closed expect delays. Watch for emergency crews. Between Sage Street (near Lamar) and County Road R (6 miles north of the Campo area) from Mile Point 74 to Mile Point 17. Road closed expect delays due to a crash. Comment: Both directions closed expect delays. Watch for emergency crews., Street US-287, City Lamar, County Prowers, State CO, ZipCode 81052, Timezone US/Mountain, Airport Code KLAA, Temperature (F) 51.0, WindChill (F) 51.0, Humidity (%) 80.0, Pressure (inch) 26.12, Visibility (miles) 10.0, Wind Direction E, Wind Speed (mph) 13.0, Precipitation (inch) 0.0, Weather Condition Cloudy, Sunrise/Sunset Day, Civil Twilight Day, Nautical Twilight Day, Astronomical Twilight Day, Start_Time_hour 12, Start_Time_month 5, Weather_Timestamp_hour 11, Weather_Timestamp_month 5",
    "Accident ID A-5968770, Source Source1, Start Latitude 37.2098886964931, Start Longitude -105.97833752581606, End Latitude 37.2255786964931, End Longitude -105.97419752581604, Accident extent (miles) 1.1079999999999999, Description Between County Road S and County Road T (1 mile north of the Romeo area) at Mile Point 16. Road closed expect delays due to a crash. Slower speeds are advised. Detour in place. Comment: Watch for detours onto County Road T5 Between County Road T and County Road S (1 mile north of the Romeo area) at Mile Point 16. Road closed expect delays due to a crash. Slower speeds are advised. Detour in place. Comment: Watch for detours onto County Road T5 , Street  County Road S, City La Jara, County Conejos, State CO, ZipCode 81140, Timezone US/Mountain, Airport Code KALS, Temperature (F) 55.0, WindChill (F) 55.0, Humidity (%) 24.0, Pressure (inch) 22.89, Visibility (miles) 10.0, Wind Direction CALM, Wind Speed (mph) 0.0, Precipitation (inch) 0.0, Weather Condition Fair, Sunrise/Sunset Day, Civil Twilight Day, Nautical Twilight Day, Astronomical Twilight Day, Start_Time_hour 15, Start_Time_month 10, Weather_Timestamp_hour 14, Weather_Timestamp_month 10",
    "Accident ID A-6137133, Source Source1, Start Latitude 40.954060999999996, Start Longitude -104.795343, End Latitude 40.950874, End Longitude -104.795581, Accident extent (miles) 0.221, Description Between County Road 128 and the Wyoming Border (13 miles north of the Nunn area) at Mile Point 306. Road closed expect delays due to a crash. Comment: US-85 closed in both directions at MP 306 4 miles South of Wyoming border.Between the Wyoming Border and County Road 128 (13 miles north of the Nunn area) at Mile Point 306. Road closed expect delays due to a crash. Comment: US-85 closed in both directions at MP 306 4 miles South of Wyoming border., Street  US Highway 85, City Carr, County Weld, State CO, ZipCode 80612-9201, Timezone US/Mountain, Airport Code KFEW, Temperature (F) 51.0, WindChill (F) 51.0, Humidity (%) 41.0, Pressure (inch) 23.88, Visibility (miles) 10.0, Wind Direction W, Wind Speed (mph) 16.0, Precipitation (inch) 0.0, Weather Condition Fair, Sunrise/Sunset Night, Civil Twilight Night, Nautical Twilight Night, Astronomical Twilight Night, Start_Time_hour 3, Start_Time_month 10, Weather_Timestamp_hour 2, Weather_Timestamp_month 10",
    "Accident ID A-4428281, Source Source1, Start Latitude 38.53234648686155, Start Longitude -104.96566414783017, End Latitude 38.47815648686155, End Longitude -104.97256414783016, Accident extent (miles) 3.763, Description Between County Road F45 and Barrett Road (10 miles north of the Florence area) at Mile Point 22. Road closed expect delays due to a crash. Comment: Watch for emergency responders. All lanes closed NB/SB. Expect delays. Between Barrett Road and County Road F45 (10 miles north of the Florence area) at Mile Point 22. Road closed expect delays due to a crash. Comment: Watch for emergency responders. All lanes closed NB/SB. Expect delays., Street  CO-115, City Penrose, County Fremont, State CO, ZipCode 81240, Timezone US/Mountain, Airport Code K1V6, Temperature (F) 63.0, WindChill (F) 63.0, Humidity (%) 27.0, Pressure (inch) 24.55, Visibility (miles) 10.0, Wind Direction SE, Wind Speed (mph) 6.0, Precipitation (inch) 0.0, Weather Condition Fair, Sunrise/Sunset Day, Civil Twilight Day, Nautical Twilight Day, Astronomical Twilight Day, Start_Time_hour 12, Start_Time_month 4, Weather_Timestamp_hour 12, Weather_Timestamp_month 4",
    "Accident ID A-4732415, Source Source1, Start Latitude 38.779571999999995, Start Longitude -107.950768, End Latitude 38.77187, End Longitude -107.96673600000001, Accident extent (miles) 1.011, Description Between 2100 Road and Lobdell Road (near Orchard City) from Mile Point 6 to Mile Point 7. Road closed expect delays due to a crash. Detour in place. Comment: Detour in place onto 875 turn left on 2200 road.Between Lobdell Road and 2100 Road (near Orchard City) from Mile Point 7 to Mile Point 6. Road closed expect delays due to a crash. Detour in place. Comment: Detour in place onto 875 turn left on 2200 road., Street  Highway 92, City Austin, County Delta, State CO, ZipCode 81410, Timezone US/Mountain, Airport Code KAJZ, Temperature (F) 19.0, WindChill (F) 11.0, Humidity (%) 77.0, Pressure (inch) 24.98, Visibility (miles) 10.0, Wind Direction N, Wind Speed (mph) 6.0, Precipitation (inch) 0.0, Weather Condition Partly Cloudy, Stop, Sunrise/Sunset Day, Civil Twilight Day, Nautical Twilight Day, Astronomical Twilight Day, Start_Time_hour 7, Start_Time_month 1, Weather_Timestamp_hour 7, Weather_Timestamp_month 1",
    "Accident ID A-6060568, Source Source1, Start Latitude 39.740563, Start Longitude -105.38991599999999, End Latitude 39.745005, End Longitude -105.39703200000001, Accident extent (miles) 0.48700000000000004, Description Between US 40; US 6 and CO 58; CO 93 (near Clear Creek Canyon) from Mile Point 260 to Mile Point 271. Road closed expect delays due to safety concerns. Comment: Due to crash. Use alternate route.Between CO 58; CO 93 and US 40; US 6 (near Clear Creek Canyon) from Mile Point 271 to Mile Point 260. Road closed expect delays due to safety concerns. Comment: Due to crash. Use alternate route., Street  US Highway 6, City Golden, County Jefferson, State CO, ZipCode 80403, Timezone US/Mountain, Airport Code KBDU, Temperature (F) 30.0, WindChill (F) 25.0, Humidity (%) 93.0, Pressure (inch) 24.35, Visibility (miles) 2.0, Wind Direction NE, Wind Speed (mph) 5.0, Precipitation (inch) 0.0, Weather Condition Light Snow, Sunrise/Sunset Day, Civil Twilight Day, Nautical Twilight Day, Astronomical Twilight Day, Start_Time_hour 8, Start_Time_month 12, Weather_Timestamp_hour 8, Weather_Timestamp_month 12",
    "Accident ID A-3996497, Source Source1, Start Latitude 36.285327, Start Longitude -86.271743, End Latitude 36.285496, End Longitude -86.271598, Accident extent (miles) 0.013999999999999999, Description State Route 10 northbound in Wilson County - Crash - Multiple Vehicles at PHILADELPHIA RD. reported at 8:12 PM 10/1/2022 and estimated to be cleared by 12:00 AM 10/2/2022 (Central Time Zone). Northbound traffic is affected with roadway closed. Southbound traffic is affected with delays expected. Northbound traffic is being diverted to alternate routes. Information provided by THP., Street  Hunters Point Pike, City Lebanon, County Wilson, State TN, ZipCode 37087-1129, Timezone US/Central, Airport Code KMQY, Temperature (F) 69.0, WindChill (F) 69.0, Humidity (%) 29.0, Pressure (inch) 29.38, Visibility (miles) 10.0, Wind Direction NNW, Wind Speed (mph) 7.0, Precipitation (inch) 0.0, Weather Condition Fair, Sunrise/Sunset Night, Civil Twilight Night, Nautical Twilight Night, Astronomical Twilight Night, Start_Time_hour 20, Start_Time_month 10, Weather_Timestamp_hour 19, Weather_Timestamp_month 10"
]

ACTIONS = r"""
# List of possible actions
actions = [
    "Deploy IRV",
    "Temporary Lane Closure",
    "Use VMS to Warn Drivers",
    "Notify Police & EMS",
    "Quick Clearance Policy",
    "Use VMS & Social Media",
    "Full or Partial Lane Closures",
    "Divert Traffic to Detour Routes",
    "Activate EOC",
    "Full Road Closure"
]
"""

PROMPT = r"""
Provide a binary array of actions to resolve the incident, based on ChatGPT4 API output: Traffic Incident Mitigation Strategies.
"""

RES =""
for INC in INCIDENTS:
    res = Q3(STRATEGY + INC + '\n\nIncident record: ' + ACTIONS + '\n\n'+PROMPT)
    print(res)
    RES+=INC+":\n\n"+res+"\n\n"

INFO:__main__:Processing question with model gpt-3.5-turbo.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Received response from OpenAI.
INFO:__main__:Processing question with model gpt-3.5-turbo.


To provide a binary array of actions based on the given incident record and the listed possible actions, we can map each action to a binary value representing its presence or absence in the incident resolution plan. 

Here is the binary array for the incident record:

\[ \text{Binary Array} = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1] \]

This binary array corresponds to the presence (1) or absence (0) of each action in the incident resolution plan for the given incident record.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Received response from OpenAI.
INFO:__main__:Processing question with model gpt-3.5-turbo.


Sure! Based on the incident record provided and the list of possible actions from the ChatGPT4 API output for Traffic Incident Mitigation Strategies, here is a binary array indicating which actions should be taken to resolve the incident:

```
[1, 1, 1, 1, 1, 0, 0, 0, 0, 1]
```

In this binary array:
- `1` indicates that the action should be taken.
- `0` indicates that the action should not be taken.

So, the actions to be taken to resolve the incident are:
1. Deploy IRV
2. Temporary Lane Closure
3. Use VMS to Warn Drivers
4. Notify Police & EMS
5. Quick Clearance Policy
10. Full Road Closure

These actions are chosen based on their relevance to the incident description provided.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Received response from OpenAI.
INFO:__main__:Processing question with model gpt-3.5-turbo.


...

To provide a binary array of actions to resolve the specific incident mentioned in the context of the ChatGPT4 API output on traffic incident mitigation strategies, we can map the listed possible actions to the ones relevant for the incident described in the context.

Here are the actions relevant to the incident:

- Deploy IRV (Incident Response Vehicle)
- Temporary Lane Closure
- Use VMS to Warn Drivers
- Notify Police & EMS
- Divert Traffic to Detour Routes
- Full Road Closure

Now, converting these actions into a binary array format:

- Deploy IRV (Incident Response Vehicle) - 1
- Temporary Lane Closure - 1
- Use VMS to Warn Drivers - 1
- Notify Police & EMS - 1
- Divert Traffic to Detour Routes - 1
- Full Road Closure - 1

The binary array representing these actions would be: [1, 1, 1, 1, 1, 1]


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Received response from OpenAI.
INFO:__main__:Processing question with model gpt-3.5-turbo.


To provide a binary array of actions to resolve the incident based on the provided ChatGPT4 API output on Traffic Incident Mitigation Strategies, we will refer to the list of possible actions and create a binary array where 1 indicates the action is required and 0 indicates the action is not required for the specific incident (Accident ID A-6060568) described in the input data.

From the given list of possible actions:
- Deploy IRV: Required
- Temporary Lane Closure: Required
- Use VMS to Warn Drivers: Required
- Notify Police & EMS: Not Required
- Quick Clearance Policy: Not Required
- Use VMS & Social Media: Not Required
- Full or Partial Lane Closures: Required
- Divert Traffic to Detour Routes: Required
- Activate EOC: Not Required
- Full Road Closure: Required

Based on this data, the binary array representing the required actions for the specific incident is:
[1, 1, 1, 0, 0, 0, 1, 1, 0, 1]


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Received response from OpenAI.


To provide a binary array of actions to resolve the incident based on the provided information, we need to match the incident type, severity, and location with the corresponding actions recommended in the table of Traffic Incident Mitigation Strategies.

Looking at the incident record and the list of possible actions, we can suggest the following binary array for actions that could be relevant in this scenario:

\[
\text{Binary Array of Actions: } [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
\]

In this binary array:
- 1 denotes that the corresponding action is recommended in the scenario.
- 0 would mean that the action is not recommended.

This binary array represents the recommended actions based on the incident details provided and the mitigation strategies outlined in the table.


# Gemini - Pro

In [51]:
STRATEGY=r"""\begin{table}[h]
\centering
\scriptsize
\caption{ChatGPT4 API output: Traffic Incident Mitigation Strategies}
\label{tab:traffic_incident_mitigation}
\begin{tabular}{|p{1cm}|p{2cm}|p{1cm}|p{3cm}|p{4cm}|p{4cm}|}
\hline
\textbf{Scenario ID} & \textbf{Incident Type} & \textbf{Severity} & \textbf{Location} & \textbf{Action} & \textbf{Equipment/Technology Required} \\
\hline
1 & Vehicle Breakdown & Low & Highway/Freeway Lane & - Deploy Incident Response Vehicle\newline - Temporary lane closure\newline - Use VMS to warn drivers & - Service Patrol Vehicle\newline - Variable Message Signs (VMS)\newline - Traffic Cones or Barriers \\
\hline
2 & Minor Two-Car Collision & Moderate & Urban Arterial & - Notify Police \& EMS if needed\newline - Quick clearance policy\newline - Use VMS \& social media & - Police/EMS Vehicles\newline - Quick Clearance Equipment (e.g., tow truck)\newline - Variable Message Signs\newline - Social Media Platforms \\
\hline
3 & Major Multi-Vehicle Crash & High & Highway/Freeway & - Full or partial lane closures\newline - Divert traffic to detour routes\newline - Activate EOC & - Police/EMS/Fire Services\newline - Emergency Operations Center (EOC) Activation\newline - Detour Signage \\
\hline
4 & Hazardous Material Spill & High & Near Urban Area & - Full road closure\newline - Mandatory evacuation if necessary\newline - HazMat Team dispatch & - HazMat Team\newline - Road Closure Signage\newline - Emergency Alert System (EAS) \\
\hline
5 & Overturned Truck & Moderate & Highway On-ramp/Off-ramp & - Partial ramp closure\newline - Speed limit reduction in area\newline - Deploy tow truck and cleanup crew & - Tow Trucks\newline - Cleanup Crew \newline - Speed Limit Signs \\
\hline
6 & Pedestrian Accident & High & Urban Crosswalk & - Full closure of affected lanes\newline - EMS priority dispatch\newline - Investigative procedures & - EMS Vehicles\newline - Police Investigation Unit\newline - Temporary Signage \\
\hline
7 & Wildlife on Road & Low & Rural Road & - Temporary speed limit reduction\newline - Use VMS to warn drivers\newline - Wildlife control dispatch & - Variable Message Signs\newline - Wildlife Control Services \\
\hline
8 & Infrastructure Failure (Bridge) & Very High & Bridge & - Full bridge closure\newline - Long-term detour setup\newline - Structural assessment & - Structural Engineering Team\newline - Permanent Detour Signage\newline - Media Briefing Equipment \\
\hline
9 & Snow/Ice Conditions & Variable & Major Roadways & - Speed limit reduction\newline - Deploy snow plows/salt trucks\newline - Use VMS and radio to inform & - Snow Plows/Salt Trucks\newline - Variable Message Signs\newline - Radio Broadcast System \\
\hline
10 & Fog/Visibility Issues & Variable & Highway/Freeway & - Speed limit reduction\newline - Flashing lights to warn drivers\newline - Use VMS to advise caution & - Variable Message Signs\newline - Highway Flashing Lights \\
\hline
\end{tabular}
\label{tab:guide}
\end{table}
"""

INCIDENTS = [
    "Accident ID A-4259643, Source Source1, Start Latitude 40.821863, Start Longitude -102.81153, End Latitude 40.815714, End Longitude -102.832031, Accident extent (miles) 1.153, Description Between US 6 (Sterling) and the Nebraska Border (4 miles east of Julesburg) from Mile Point 125 to Mile Point 184.14. Road closed expect delays due to a crash. Comment: Highway closed from Sterling to Nebraska state line due to adverse driving conditions and multiple crashes Between the Nebraska Border (4 miles east of Julesburg) and US 6 (Sterling) from Mile Point 184.14 to Mile Point 125. Road closed expect delays due to a crash. Comment: Highway closed from Sterling to Nebraska state line due to adverse driving conditions and multiple crashes, Street I-76, City Crook, County Logan, State CO, ZipCode 80726, Timezone US/Mountain, Airport Code KSNY, Temperature (F) 17.0, WindChill (F) -1.0, Humidity (%) 74.0, Pressure (inch) 25.51, Visibility (miles) 10.0, Wind Direction WNW, Wind Speed (mph) 25.0, Precipitation (inch) 0.0, Weather Condition Fair / Windy, Sunrise/Sunset Night, Civil Twilight Day, Nautical Twilight Day, Astronomical Twilight Day, Start_Time_hour 16, Start_Time_month 12, Weather_Timestamp_hour 16, Weather_Timestamp_month 12",
    "Accident ID A-5128843, Source Source1, Start Latitude 40.50956368427033, Start Longitude -103.80330204914186, End Latitude 40.49505368427033, End Longitude -103.80324204914186, Accident extent (miles) 1.003, Description Between County Road II and County Road 76 (6 miles west of the Raymer area) at Mile Point 104. Road closed expect delays due to a crash. Comment: CO52 closed both directions at MP104 about 7 miles south of CO14 due to crash alternate route advised. Between County Road 76 and County Road II (6 miles west of the Raymer area) at Mile Point 104. Road closed expect delays due to a crash. Comment: CO52 closed both directions at MP104 about 7 miles south of CO14 due to crash alternate route advised., Street  County Road JJ, City Fort Morgan, County Morgan, State CO, ZipCode 80701-9608, Timezone US/Mountain, Airport Code KFMM, Temperature (F) 34.0, WindChill (F) 27.0, Humidity (%) 70.0, Pressure (inch) 25.36, Visibility (miles) 10.0, Wind Direction WSW, Wind Speed (mph) 8.0, Precipitation (inch) 0.0, Weather Condition Fair, Stop, Sunrise/Sunset Day, Civil Twilight Day, Nautical Twilight Day, Astronomical Twilight Day, Start_Time_hour 8, Start_Time_month 3, Weather_Timestamp_hour 8, Weather_Timestamp_month 3",
    "Accident ID A-4227983, Source Source1, Start Latitude 37.762206, Start Longitude -104.835217, End Latitude 37.956072, End Longitude -104.800249, Accident extent (miles) 13.53, Description Between Exit 59: Butte Road (5 miles north of the Walsenburg area) and CO 165 (16 miles south of the Pueblo area) from Mile Point 60 to Mile Point 74. Road closed expect delays due to a crash. Slower speeds are advised. Comment: Seek alternate route use caution Between CO 165 (16 miles south of the Pueblo area) and Exit 59: Butte Road (5 miles north of the Walsenburg area) from Mile Point 74 to Mile Point 60. Road closed expect delays due to a crash. Slower speeds are advised. Comment: Seek alternate route use caution, Street I-25 N, City Walsenburg, County Huerfano, State CO, ZipCode 81089, Timezone US/Mountain, Airport Code KVTP, Temperature (F) 27.0, WindChill (F) 16.0, Humidity (%) 86.0, Pressure (inch) 20.32, Visibility (miles) 1.0, Wind Direction WSW, Wind Speed (mph) 13.0, Precipitation (inch) 0.0, Weather Condition Light Snow, Sunrise/Sunset Night, Civil Twilight Night, Nautical Twilight Night, Astronomical Twilight Night, Start_Time_hour 22, Start_Time_month 4, Weather_Timestamp_hour 22, Weather_Timestamp_month 4",
    "Accident ID A-4888575, Source Source1, Start Latitude 37.665370702563926, Start Longitude -102.62025475453079, End Latitude 37.70185070256392, End Longitude -102.60910475453079, Accident extent (miles) 2.593, Description Between County Road R (6 miles north of the Campo area) and Sage Street (near Lamar) from Mile Point 17 to Mile Point 74. Road closed expect delays due to a crash. Comment: Both directions closed expect delays. Watch for emergency crews. Between Sage Street (near Lamar) and County Road R (6 miles north of the Campo area) from Mile Point 74 to Mile Point 17. Road closed expect delays due to a crash. Comment: Both directions closed expect delays. Watch for emergency crews., Street US-287, City Lamar, County Prowers, State CO, ZipCode 81052, Timezone US/Mountain, Airport Code KLAA, Temperature (F) 51.0, WindChill (F) 51.0, Humidity (%) 80.0, Pressure (inch) 26.12, Visibility (miles) 10.0, Wind Direction E, Wind Speed (mph) 13.0, Precipitation (inch) 0.0, Weather Condition Cloudy, Sunrise/Sunset Day, Civil Twilight Day, Nautical Twilight Day, Astronomical Twilight Day, Start_Time_hour 12, Start_Time_month 5, Weather_Timestamp_hour 11, Weather_Timestamp_month 5",
    "Accident ID A-5968770, Source Source1, Start Latitude 37.2098886964931, Start Longitude -105.97833752581606, End Latitude 37.2255786964931, End Longitude -105.97419752581604, Accident extent (miles) 1.1079999999999999, Description Between County Road S and County Road T (1 mile north of the Romeo area) at Mile Point 16. Road closed expect delays due to a crash. Slower speeds are advised. Detour in place. Comment: Watch for detours onto County Road T5 Between County Road T and County Road S (1 mile north of the Romeo area) at Mile Point 16. Road closed expect delays due to a crash. Slower speeds are advised. Detour in place. Comment: Watch for detours onto County Road T5 , Street  County Road S, City La Jara, County Conejos, State CO, ZipCode 81140, Timezone US/Mountain, Airport Code KALS, Temperature (F) 55.0, WindChill (F) 55.0, Humidity (%) 24.0, Pressure (inch) 22.89, Visibility (miles) 10.0, Wind Direction CALM, Wind Speed (mph) 0.0, Precipitation (inch) 0.0, Weather Condition Fair, Sunrise/Sunset Day, Civil Twilight Day, Nautical Twilight Day, Astronomical Twilight Day, Start_Time_hour 15, Start_Time_month 10, Weather_Timestamp_hour 14, Weather_Timestamp_month 10",
    "Accident ID A-6137133, Source Source1, Start Latitude 40.954060999999996, Start Longitude -104.795343, End Latitude 40.950874, End Longitude -104.795581, Accident extent (miles) 0.221, Description Between County Road 128 and the Wyoming Border (13 miles north of the Nunn area) at Mile Point 306. Road closed expect delays due to a crash. Comment: US-85 closed in both directions at MP 306 4 miles South of Wyoming border.Between the Wyoming Border and County Road 128 (13 miles north of the Nunn area) at Mile Point 306. Road closed expect delays due to a crash. Comment: US-85 closed in both directions at MP 306 4 miles South of Wyoming border., Street  US Highway 85, City Carr, County Weld, State CO, ZipCode 80612-9201, Timezone US/Mountain, Airport Code KFEW, Temperature (F) 51.0, WindChill (F) 51.0, Humidity (%) 41.0, Pressure (inch) 23.88, Visibility (miles) 10.0, Wind Direction W, Wind Speed (mph) 16.0, Precipitation (inch) 0.0, Weather Condition Fair, Sunrise/Sunset Night, Civil Twilight Night, Nautical Twilight Night, Astronomical Twilight Night, Start_Time_hour 3, Start_Time_month 10, Weather_Timestamp_hour 2, Weather_Timestamp_month 10",
    "Accident ID A-4428281, Source Source1, Start Latitude 38.53234648686155, Start Longitude -104.96566414783017, End Latitude 38.47815648686155, End Longitude -104.97256414783016, Accident extent (miles) 3.763, Description Between County Road F45 and Barrett Road (10 miles north of the Florence area) at Mile Point 22. Road closed expect delays due to a crash. Comment: Watch for emergency responders. All lanes closed NB/SB. Expect delays. Between Barrett Road and County Road F45 (10 miles north of the Florence area) at Mile Point 22. Road closed expect delays due to a crash. Comment: Watch for emergency responders. All lanes closed NB/SB. Expect delays., Street  CO-115, City Penrose, County Fremont, State CO, ZipCode 81240, Timezone US/Mountain, Airport Code K1V6, Temperature (F) 63.0, WindChill (F) 63.0, Humidity (%) 27.0, Pressure (inch) 24.55, Visibility (miles) 10.0, Wind Direction SE, Wind Speed (mph) 6.0, Precipitation (inch) 0.0, Weather Condition Fair, Sunrise/Sunset Day, Civil Twilight Day, Nautical Twilight Day, Astronomical Twilight Day, Start_Time_hour 12, Start_Time_month 4, Weather_Timestamp_hour 12, Weather_Timestamp_month 4",
    "Accident ID A-4732415, Source Source1, Start Latitude 38.779571999999995, Start Longitude -107.950768, End Latitude 38.77187, End Longitude -107.96673600000001, Accident extent (miles) 1.011, Description Between 2100 Road and Lobdell Road (near Orchard City) from Mile Point 6 to Mile Point 7. Road closed expect delays due to a crash. Detour in place. Comment: Detour in place onto 875 turn left on 2200 road.Between Lobdell Road and 2100 Road (near Orchard City) from Mile Point 7 to Mile Point 6. Road closed expect delays due to a crash. Detour in place. Comment: Detour in place onto 875 turn left on 2200 road., Street  Highway 92, City Austin, County Delta, State CO, ZipCode 81410, Timezone US/Mountain, Airport Code KAJZ, Temperature (F) 19.0, WindChill (F) 11.0, Humidity (%) 77.0, Pressure (inch) 24.98, Visibility (miles) 10.0, Wind Direction N, Wind Speed (mph) 6.0, Precipitation (inch) 0.0, Weather Condition Partly Cloudy, Stop, Sunrise/Sunset Day, Civil Twilight Day, Nautical Twilight Day, Astronomical Twilight Day, Start_Time_hour 7, Start_Time_month 1, Weather_Timestamp_hour 7, Weather_Timestamp_month 1",
    "Accident ID A-6060568, Source Source1, Start Latitude 39.740563, Start Longitude -105.38991599999999, End Latitude 39.745005, End Longitude -105.39703200000001, Accident extent (miles) 0.48700000000000004, Description Between US 40; US 6 and CO 58; CO 93 (near Clear Creek Canyon) from Mile Point 260 to Mile Point 271. Road closed expect delays due to safety concerns. Comment: Due to crash. Use alternate route.Between CO 58; CO 93 and US 40; US 6 (near Clear Creek Canyon) from Mile Point 271 to Mile Point 260. Road closed expect delays due to safety concerns. Comment: Due to crash. Use alternate route., Street  US Highway 6, City Golden, County Jefferson, State CO, ZipCode 80403, Timezone US/Mountain, Airport Code KBDU, Temperature (F) 30.0, WindChill (F) 25.0, Humidity (%) 93.0, Pressure (inch) 24.35, Visibility (miles) 2.0, Wind Direction NE, Wind Speed (mph) 5.0, Precipitation (inch) 0.0, Weather Condition Light Snow, Sunrise/Sunset Day, Civil Twilight Day, Nautical Twilight Day, Astronomical Twilight Day, Start_Time_hour 8, Start_Time_month 12, Weather_Timestamp_hour 8, Weather_Timestamp_month 12",
    "Accident ID A-3996497, Source Source1, Start Latitude 36.285327, Start Longitude -86.271743, End Latitude 36.285496, End Longitude -86.271598, Accident extent (miles) 0.013999999999999999, Description State Route 10 northbound in Wilson County - Crash - Multiple Vehicles at PHILADELPHIA RD. reported at 8:12 PM 10/1/2022 and estimated to be cleared by 12:00 AM 10/2/2022 (Central Time Zone). Northbound traffic is affected with roadway closed. Southbound traffic is affected with delays expected. Northbound traffic is being diverted to alternate routes. Information provided by THP., Street  Hunters Point Pike, City Lebanon, County Wilson, State TN, ZipCode 37087-1129, Timezone US/Central, Airport Code KMQY, Temperature (F) 69.0, WindChill (F) 69.0, Humidity (%) 29.0, Pressure (inch) 29.38, Visibility (miles) 10.0, Wind Direction NNW, Wind Speed (mph) 7.0, Precipitation (inch) 0.0, Weather Condition Fair, Sunrise/Sunset Night, Civil Twilight Night, Nautical Twilight Night, Astronomical Twilight Night, Start_Time_hour 20, Start_Time_month 10, Weather_Timestamp_hour 19, Weather_Timestamp_month 10"
]

ACTIONS = r"""
# List of possible actions
actions = [
    "Deploy IRV",
    "Temporary Lane Closure",
    "Use VMS to Warn Drivers",
    "Notify Police & EMS",
    "Quick Clearance Policy",
    "Use VMS & Social Media",
    "Full or Partial Lane Closures",
    "Divert Traffic to Detour Routes",
    "Activate EOC",
    "Full Road Closure"
]
"""

PROMPT = r"""
Provide a binary array of actions to resolve the incident, based on ChatGPT4 API output: Traffic Incident Mitigation Strategies.
"""

RES =""
for INC in INCIDENTS:
    res = GP(STRATEGY + INC + '\n\nIncident record: ' + ACTIONS + '\n\n'+PROMPT)
    print(res)
    RES+=INC+":\n\n"+res+"\n\n" 

INFO:__main__:Processing question with model gemini-1.5-pro.
INFO:__main__:Received response from Google Generative AI.
INFO:__main__:Processing question with model gemini-1.5-pro.


Here's how we can break down the incident and determine the appropriate actions:

**Incident Analysis**

* **Incident Type:** Multiple crashes due to adverse weather (snow/ice likely implied)
* **Severity:** High (road closure indicated) 
* **Location:** Highway/Freeway (I-76)
* **Extent:** Significant portion of highway closed (Sterling, CO to Nebraska border)

**Recommended Actions (Binary Array)**

Based on the table and incident details, here's a likely binary representation:

```
actions = [
    "Deploy IRV",              # 1 Likely needed for assisting motorists 
    "Temporary Lane Closure",   # 1  Full closure is in effect, so temporary closures are inherent
    "Use VMS to Warn Drivers",  # 1  Crucial for informing about closures and detours
    "Notify Police & EMS",     # 1  Multiple crashes necessitate emergency response
    "Quick Clearance Policy",   # 1  Essential to reopen the road ASAP
    "Use VMS & Social Media",  # 1  Widespread communication is vital due to closure

INFO:__main__:Received response from Google Generative AI.
INFO:__main__:Processing question with model gemini-1.5-pro.


Here's a binary array representing the recommended actions based on the provided information and  table "ChatGPT4 API output: Traffic Incident Mitigation Strategies". 

**Incident Analysis:**

* **Incident Type:** Crash (appears to be major based on road closure)
* **Severity:** High (Road Closed, alternate routes advised)
* **Location:**  Highway/Freeway (CO52)

**Recommended Actions Binary Array (1=Action Needed, 0=Action Not Needed):**

[1, 1, 1, 1, 1, 1, 1, 1, 1, 0]

**Explanation:**

1. **Deploy IRV (1):**  Yes, an Incident Response Vehicle would be valuable for on-scene assistance and traffic management.
2. **Temporary Lane Closure (1):**  Likely necessary initially to assess the situation and begin clearing the accident.
3. **Use VMS to Warn Drivers (1):** Crucial for alerting approaching drivers of the crash and potential delays.
4. **Notify Police & EMS (1):** Absolutely, given the description suggesting a significant crash ("road closed").
5. **Quick Clearance Policy (1):**  

INFO:__main__:Received response from Google Generative AI.
INFO:__main__:Processing question with model gemini-1.5-pro.


...

Here's how we can determine the appropriate actions based on the provided information and table:

**Analyzing the Incident Record:**

* **Incident Type:** Crash (explicitly stated)
* **Severity:** High (Road closed, detour in place suggests significant impact)
* **Location:** Highway 92 (classified as Highway/Freeway)

**Matching to Scenario IDs in the Table:**

* **Scenario 1:** Vehicle Breakdown - Not a match
* **Scenario 2:** Minor Collision - Severity doesn't match
* **Scenario 3:** Major Multi-Vehicle Crash -  **Strong match** (Incident type, severity, and location align) 

**Let's be conservative and also consider:**

* **Scenario 5:** Overturned Truck -  *Possible* match, as we don't know the exact cause of the crash. Overturned trucks often lead to major road closures.

**Actions from Matching Scenarios:**

* **Scenario 3:** 
    * Full or Partial Lane Closures - **YES** (Mentioned in the description)
    * Divert Traffic to Detour Routes - **YES** (Detour in place)
    * Activ

INFO:__main__:Received response from Google Generative AI.
INFO:__main__:Processing question with model gemini-1.5-pro.


Here's a binary array representing recommended actions based on the provided information and the ChatGPT4 table:

**Incident Analysis:**

* **Incident Type:** Major accident (likely multi-vehicle given road closure extent)
* **Severity:** High (road closure, safety concerns)
* **Location:** Highway/Freeway (US Highway 6)

**Recommended Actions based on Table Analysis (matching scenarios 3 & potentially elements of 1):**

```
actions = [
    "Deploy IRV",           # 1 - Likely, as this is standard for highway incidents
    "Temporary Lane Closure", # 1 - Potentially, depends on severity & if any lanes open
    "Use VMS to Warn Drivers", # 1, 3 - YES, essential for warning & detour info
    "Notify Police & EMS",    # 2, 3 - YES, highly likely given "safety concerns"
    "Quick Clearance Policy",  # 2 -  Likely, to reopen road ASAP 
    "Use VMS & Social Media",  # 2 - YES, VMS for immediate, Social Media for wider reach
    "Full or Partial Lane Closures", # 3 - YES, description implie

INFO:__main__:Received response from Google Generative AI.


Here's how we can break down the incident and determine the appropriate actions:

**Incident Analysis**

* **Incident Type:** Multi-vehicle crash 
* **Location:** Highway/Freeway (State Route 10)
* **Severity:** High (Roadway closed, traffic diverted, significant delays)

**Action Mapping (Based on Table 1)**

Referring to your provided table "ChatGPT4 API output: Traffic Incident Mitigation Strategies," the most relevant scenarios appear to be:

* **Scenario 2 (Minor Two-Car Collision):**  While this incident is more severe, some actions still apply (police/EMS, quick clearance).
* **Scenario 3 (Major Multi-Vehicle Crash):** This is the closest match, informing our primary actions.

**Binary Action Array**

Here's a binary representation of the recommended actions, where '1' indicates the action is needed, and '0' indicates it's not:

```
actions = [
    "Deploy IRV",              # 1 - Likely needed for initial response and assistance
    "Temporary Lane Closure",   # 1 - Incident me

# ChatGPT 4o

In [53]:
STRATEGY=r"""\begin{table}[h]
\centering
\scriptsize
\caption{ChatGPT4 API output: Traffic Incident Mitigation Strategies}
\label{tab:traffic_incident_mitigation}
\begin{tabular}{|p{1cm}|p{2cm}|p{1cm}|p{3cm}|p{4cm}|p{4cm}|}
\hline
\textbf{Scenario ID} & \textbf{Incident Type} & \textbf{Severity} & \textbf{Location} & \textbf{Action} & \textbf{Equipment/Technology Required} \\
\hline
1 & Vehicle Breakdown & Low & Highway/Freeway Lane & - Deploy Incident Response Vehicle\newline - Temporary lane closure\newline - Use VMS to warn drivers & - Service Patrol Vehicle\newline - Variable Message Signs (VMS)\newline - Traffic Cones or Barriers \\
\hline
2 & Minor Two-Car Collision & Moderate & Urban Arterial & - Notify Police \& EMS if needed\newline - Quick clearance policy\newline - Use VMS \& social media & - Police/EMS Vehicles\newline - Quick Clearance Equipment (e.g., tow truck)\newline - Variable Message Signs\newline - Social Media Platforms \\
\hline
3 & Major Multi-Vehicle Crash & High & Highway/Freeway & - Full or partial lane closures\newline - Divert traffic to detour routes\newline - Activate EOC & - Police/EMS/Fire Services\newline - Emergency Operations Center (EOC) Activation\newline - Detour Signage \\
\hline
4 & Hazardous Material Spill & High & Near Urban Area & - Full road closure\newline - Mandatory evacuation if necessary\newline - HazMat Team dispatch & - HazMat Team\newline - Road Closure Signage\newline - Emergency Alert System (EAS) \\
\hline
5 & Overturned Truck & Moderate & Highway On-ramp/Off-ramp & - Partial ramp closure\newline - Speed limit reduction in area\newline - Deploy tow truck and cleanup crew & - Tow Trucks\newline - Cleanup Crew \newline - Speed Limit Signs \\
\hline
6 & Pedestrian Accident & High & Urban Crosswalk & - Full closure of affected lanes\newline - EMS priority dispatch\newline - Investigative procedures & - EMS Vehicles\newline - Police Investigation Unit\newline - Temporary Signage \\
\hline
7 & Wildlife on Road & Low & Rural Road & - Temporary speed limit reduction\newline - Use VMS to warn drivers\newline - Wildlife control dispatch & - Variable Message Signs\newline - Wildlife Control Services \\
\hline
8 & Infrastructure Failure (Bridge) & Very High & Bridge & - Full bridge closure\newline - Long-term detour setup\newline - Structural assessment & - Structural Engineering Team\newline - Permanent Detour Signage\newline - Media Briefing Equipment \\
\hline
9 & Snow/Ice Conditions & Variable & Major Roadways & - Speed limit reduction\newline - Deploy snow plows/salt trucks\newline - Use VMS and radio to inform & - Snow Plows/Salt Trucks\newline - Variable Message Signs\newline - Radio Broadcast System \\
\hline
10 & Fog/Visibility Issues & Variable & Highway/Freeway & - Speed limit reduction\newline - Flashing lights to warn drivers\newline - Use VMS to advise caution & - Variable Message Signs\newline - Highway Flashing Lights \\
\hline
\end{tabular}
\label{tab:guide}
\end{table}
"""

INCIDENTS = [
    "Accident ID A-4259643, Source Source1, Start Latitude 40.821863, Start Longitude -102.81153, End Latitude 40.815714, End Longitude -102.832031, Accident extent (miles) 1.153, Description Between US 6 (Sterling) and the Nebraska Border (4 miles east of Julesburg) from Mile Point 125 to Mile Point 184.14. Road closed expect delays due to a crash. Comment: Highway closed from Sterling to Nebraska state line due to adverse driving conditions and multiple crashes Between the Nebraska Border (4 miles east of Julesburg) and US 6 (Sterling) from Mile Point 184.14 to Mile Point 125. Road closed expect delays due to a crash. Comment: Highway closed from Sterling to Nebraska state line due to adverse driving conditions and multiple crashes, Street I-76, City Crook, County Logan, State CO, ZipCode 80726, Timezone US/Mountain, Airport Code KSNY, Temperature (F) 17.0, WindChill (F) -1.0, Humidity (%) 74.0, Pressure (inch) 25.51, Visibility (miles) 10.0, Wind Direction WNW, Wind Speed (mph) 25.0, Precipitation (inch) 0.0, Weather Condition Fair / Windy, Sunrise/Sunset Night, Civil Twilight Day, Nautical Twilight Day, Astronomical Twilight Day, Start_Time_hour 16, Start_Time_month 12, Weather_Timestamp_hour 16, Weather_Timestamp_month 12",
    "Accident ID A-5128843, Source Source1, Start Latitude 40.50956368427033, Start Longitude -103.80330204914186, End Latitude 40.49505368427033, End Longitude -103.80324204914186, Accident extent (miles) 1.003, Description Between County Road II and County Road 76 (6 miles west of the Raymer area) at Mile Point 104. Road closed expect delays due to a crash. Comment: CO52 closed both directions at MP104 about 7 miles south of CO14 due to crash alternate route advised. Between County Road 76 and County Road II (6 miles west of the Raymer area) at Mile Point 104. Road closed expect delays due to a crash. Comment: CO52 closed both directions at MP104 about 7 miles south of CO14 due to crash alternate route advised., Street  County Road JJ, City Fort Morgan, County Morgan, State CO, ZipCode 80701-9608, Timezone US/Mountain, Airport Code KFMM, Temperature (F) 34.0, WindChill (F) 27.0, Humidity (%) 70.0, Pressure (inch) 25.36, Visibility (miles) 10.0, Wind Direction WSW, Wind Speed (mph) 8.0, Precipitation (inch) 0.0, Weather Condition Fair, Stop, Sunrise/Sunset Day, Civil Twilight Day, Nautical Twilight Day, Astronomical Twilight Day, Start_Time_hour 8, Start_Time_month 3, Weather_Timestamp_hour 8, Weather_Timestamp_month 3",
    "Accident ID A-4227983, Source Source1, Start Latitude 37.762206, Start Longitude -104.835217, End Latitude 37.956072, End Longitude -104.800249, Accident extent (miles) 13.53, Description Between Exit 59: Butte Road (5 miles north of the Walsenburg area) and CO 165 (16 miles south of the Pueblo area) from Mile Point 60 to Mile Point 74. Road closed expect delays due to a crash. Slower speeds are advised. Comment: Seek alternate route use caution Between CO 165 (16 miles south of the Pueblo area) and Exit 59: Butte Road (5 miles north of the Walsenburg area) from Mile Point 74 to Mile Point 60. Road closed expect delays due to a crash. Slower speeds are advised. Comment: Seek alternate route use caution, Street I-25 N, City Walsenburg, County Huerfano, State CO, ZipCode 81089, Timezone US/Mountain, Airport Code KVTP, Temperature (F) 27.0, WindChill (F) 16.0, Humidity (%) 86.0, Pressure (inch) 20.32, Visibility (miles) 1.0, Wind Direction WSW, Wind Speed (mph) 13.0, Precipitation (inch) 0.0, Weather Condition Light Snow, Sunrise/Sunset Night, Civil Twilight Night, Nautical Twilight Night, Astronomical Twilight Night, Start_Time_hour 22, Start_Time_month 4, Weather_Timestamp_hour 22, Weather_Timestamp_month 4",
    "Accident ID A-4888575, Source Source1, Start Latitude 37.665370702563926, Start Longitude -102.62025475453079, End Latitude 37.70185070256392, End Longitude -102.60910475453079, Accident extent (miles) 2.593, Description Between County Road R (6 miles north of the Campo area) and Sage Street (near Lamar) from Mile Point 17 to Mile Point 74. Road closed expect delays due to a crash. Comment: Both directions closed expect delays. Watch for emergency crews. Between Sage Street (near Lamar) and County Road R (6 miles north of the Campo area) from Mile Point 74 to Mile Point 17. Road closed expect delays due to a crash. Comment: Both directions closed expect delays. Watch for emergency crews., Street US-287, City Lamar, County Prowers, State CO, ZipCode 81052, Timezone US/Mountain, Airport Code KLAA, Temperature (F) 51.0, WindChill (F) 51.0, Humidity (%) 80.0, Pressure (inch) 26.12, Visibility (miles) 10.0, Wind Direction E, Wind Speed (mph) 13.0, Precipitation (inch) 0.0, Weather Condition Cloudy, Sunrise/Sunset Day, Civil Twilight Day, Nautical Twilight Day, Astronomical Twilight Day, Start_Time_hour 12, Start_Time_month 5, Weather_Timestamp_hour 11, Weather_Timestamp_month 5",
    "Accident ID A-5968770, Source Source1, Start Latitude 37.2098886964931, Start Longitude -105.97833752581606, End Latitude 37.2255786964931, End Longitude -105.97419752581604, Accident extent (miles) 1.1079999999999999, Description Between County Road S and County Road T (1 mile north of the Romeo area) at Mile Point 16. Road closed expect delays due to a crash. Slower speeds are advised. Detour in place. Comment: Watch for detours onto County Road T5 Between County Road T and County Road S (1 mile north of the Romeo area) at Mile Point 16. Road closed expect delays due to a crash. Slower speeds are advised. Detour in place. Comment: Watch for detours onto County Road T5 , Street  County Road S, City La Jara, County Conejos, State CO, ZipCode 81140, Timezone US/Mountain, Airport Code KALS, Temperature (F) 55.0, WindChill (F) 55.0, Humidity (%) 24.0, Pressure (inch) 22.89, Visibility (miles) 10.0, Wind Direction CALM, Wind Speed (mph) 0.0, Precipitation (inch) 0.0, Weather Condition Fair, Sunrise/Sunset Day, Civil Twilight Day, Nautical Twilight Day, Astronomical Twilight Day, Start_Time_hour 15, Start_Time_month 10, Weather_Timestamp_hour 14, Weather_Timestamp_month 10",
    "Accident ID A-6137133, Source Source1, Start Latitude 40.954060999999996, Start Longitude -104.795343, End Latitude 40.950874, End Longitude -104.795581, Accident extent (miles) 0.221, Description Between County Road 128 and the Wyoming Border (13 miles north of the Nunn area) at Mile Point 306. Road closed expect delays due to a crash. Comment: US-85 closed in both directions at MP 306 4 miles South of Wyoming border.Between the Wyoming Border and County Road 128 (13 miles north of the Nunn area) at Mile Point 306. Road closed expect delays due to a crash. Comment: US-85 closed in both directions at MP 306 4 miles South of Wyoming border., Street  US Highway 85, City Carr, County Weld, State CO, ZipCode 80612-9201, Timezone US/Mountain, Airport Code KFEW, Temperature (F) 51.0, WindChill (F) 51.0, Humidity (%) 41.0, Pressure (inch) 23.88, Visibility (miles) 10.0, Wind Direction W, Wind Speed (mph) 16.0, Precipitation (inch) 0.0, Weather Condition Fair, Sunrise/Sunset Night, Civil Twilight Night, Nautical Twilight Night, Astronomical Twilight Night, Start_Time_hour 3, Start_Time_month 10, Weather_Timestamp_hour 2, Weather_Timestamp_month 10",
    "Accident ID A-4428281, Source Source1, Start Latitude 38.53234648686155, Start Longitude -104.96566414783017, End Latitude 38.47815648686155, End Longitude -104.97256414783016, Accident extent (miles) 3.763, Description Between County Road F45 and Barrett Road (10 miles north of the Florence area) at Mile Point 22. Road closed expect delays due to a crash. Comment: Watch for emergency responders. All lanes closed NB/SB. Expect delays. Between Barrett Road and County Road F45 (10 miles north of the Florence area) at Mile Point 22. Road closed expect delays due to a crash. Comment: Watch for emergency responders. All lanes closed NB/SB. Expect delays., Street  CO-115, City Penrose, County Fremont, State CO, ZipCode 81240, Timezone US/Mountain, Airport Code K1V6, Temperature (F) 63.0, WindChill (F) 63.0, Humidity (%) 27.0, Pressure (inch) 24.55, Visibility (miles) 10.0, Wind Direction SE, Wind Speed (mph) 6.0, Precipitation (inch) 0.0, Weather Condition Fair, Sunrise/Sunset Day, Civil Twilight Day, Nautical Twilight Day, Astronomical Twilight Day, Start_Time_hour 12, Start_Time_month 4, Weather_Timestamp_hour 12, Weather_Timestamp_month 4",
    "Accident ID A-4732415, Source Source1, Start Latitude 38.779571999999995, Start Longitude -107.950768, End Latitude 38.77187, End Longitude -107.96673600000001, Accident extent (miles) 1.011, Description Between 2100 Road and Lobdell Road (near Orchard City) from Mile Point 6 to Mile Point 7. Road closed expect delays due to a crash. Detour in place. Comment: Detour in place onto 875 turn left on 2200 road.Between Lobdell Road and 2100 Road (near Orchard City) from Mile Point 7 to Mile Point 6. Road closed expect delays due to a crash. Detour in place. Comment: Detour in place onto 875 turn left on 2200 road., Street  Highway 92, City Austin, County Delta, State CO, ZipCode 81410, Timezone US/Mountain, Airport Code KAJZ, Temperature (F) 19.0, WindChill (F) 11.0, Humidity (%) 77.0, Pressure (inch) 24.98, Visibility (miles) 10.0, Wind Direction N, Wind Speed (mph) 6.0, Precipitation (inch) 0.0, Weather Condition Partly Cloudy, Stop, Sunrise/Sunset Day, Civil Twilight Day, Nautical Twilight Day, Astronomical Twilight Day, Start_Time_hour 7, Start_Time_month 1, Weather_Timestamp_hour 7, Weather_Timestamp_month 1",
    "Accident ID A-6060568, Source Source1, Start Latitude 39.740563, Start Longitude -105.38991599999999, End Latitude 39.745005, End Longitude -105.39703200000001, Accident extent (miles) 0.48700000000000004, Description Between US 40; US 6 and CO 58; CO 93 (near Clear Creek Canyon) from Mile Point 260 to Mile Point 271. Road closed expect delays due to safety concerns. Comment: Due to crash. Use alternate route.Between CO 58; CO 93 and US 40; US 6 (near Clear Creek Canyon) from Mile Point 271 to Mile Point 260. Road closed expect delays due to safety concerns. Comment: Due to crash. Use alternate route., Street  US Highway 6, City Golden, County Jefferson, State CO, ZipCode 80403, Timezone US/Mountain, Airport Code KBDU, Temperature (F) 30.0, WindChill (F) 25.0, Humidity (%) 93.0, Pressure (inch) 24.35, Visibility (miles) 2.0, Wind Direction NE, Wind Speed (mph) 5.0, Precipitation (inch) 0.0, Weather Condition Light Snow, Sunrise/Sunset Day, Civil Twilight Day, Nautical Twilight Day, Astronomical Twilight Day, Start_Time_hour 8, Start_Time_month 12, Weather_Timestamp_hour 8, Weather_Timestamp_month 12",
    "Accident ID A-3996497, Source Source1, Start Latitude 36.285327, Start Longitude -86.271743, End Latitude 36.285496, End Longitude -86.271598, Accident extent (miles) 0.013999999999999999, Description State Route 10 northbound in Wilson County - Crash - Multiple Vehicles at PHILADELPHIA RD. reported at 8:12 PM 10/1/2022 and estimated to be cleared by 12:00 AM 10/2/2022 (Central Time Zone). Northbound traffic is affected with roadway closed. Southbound traffic is affected with delays expected. Northbound traffic is being diverted to alternate routes. Information provided by THP., Street  Hunters Point Pike, City Lebanon, County Wilson, State TN, ZipCode 37087-1129, Timezone US/Central, Airport Code KMQY, Temperature (F) 69.0, WindChill (F) 69.0, Humidity (%) 29.0, Pressure (inch) 29.38, Visibility (miles) 10.0, Wind Direction NNW, Wind Speed (mph) 7.0, Precipitation (inch) 0.0, Weather Condition Fair, Sunrise/Sunset Night, Civil Twilight Night, Nautical Twilight Night, Astronomical Twilight Night, Start_Time_hour 20, Start_Time_month 10, Weather_Timestamp_hour 19, Weather_Timestamp_month 10"
]

ACTIONS = r"""
# List of possible actions
actions = [
    "Deploy IRV",
    "Temporary Lane Closure",
    "Use VMS to Warn Drivers",
    "Notify Police & EMS",
    "Quick Clearance Policy",
    "Use VMS & Social Media",
    "Full or Partial Lane Closures",
    "Divert Traffic to Detour Routes",
    "Activate EOC",
    "Full Road Closure"
]
"""

PROMPT = r"""
Provide a binary array of actions to resolve the incident, based on ChatGPT4 API output: Traffic Incident Mitigation Strategies.
"""

RES =""
for INC in INCIDENTS:
    res = Q(STRATEGY + INC + '\n\nIncident record: ' + ACTIONS + '\n\n'+PROMPT)
    print(res)
    RES+=INC+":\n\n"+res+"\n\n" 

INFO:__main__:Processing question with model gpt-4o.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Received response from OpenAI.
INFO:__main__:Processing question with model gpt-4o.


Based on the information provided for Accident ID A-4259643 and cross-referencing with the ChatGPT4 API output: Traffic Incident Mitigation Strategies, it is clear that the incident involves adverse driving conditions and multiple crashes on a highway/freeway. This aligns with Scenario ID 3: Major Multi-Vehicle Crash, with high severity on a highway.

Here is the binary array corresponding to the list of possible actions for the given incident:

1. Deploy IRV (Incident Response Vehicle) - Not necessary specifically mentioned in the table for major crashes.
2. Temporary Lane Closure - Not applicable since a full road closure is indicated.
3. Use VMS to Warn Drivers - YES
4. Notify Police & EMS - YES
5. Quick Clearance Policy - Not mentioned directly, more about detours and closures.
6. Use VMS & Social Media - YES
7. Full or Partial Lane Closures - Partial not applied, but full lane closure - YES
8. Divert Traffic to Detour Routes - YES
9. Activate EOC (Emergency Operations Center) - YE

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Received response from OpenAI.
INFO:__main__:Processing question with model gpt-4o.


Given the details of the traffic incident between County Road II and County Road 76 near Fort Morgan, CO, with a road closure and expected delays due to a crash on CO52, here is a binary array representing appropriate actions to resolve the incident, based on the provided traffic incident mitigation strategies:

1. **Deploy IRV** (Incident Response Vehicle)
2. **Temporary Lane Closure**
3. **Use VMS to Warn Drivers**
4. **Notify Police & EMS**
5. **Quick Clearance Policy**
6. **Use VMS & Social Media**
7. **Full or Partial Lane Closures**
8. **Divert Traffic to Detour Routes**
9. **Activate EOC** (Emergency Operations Center)
10. **Full Road Closure**

### Analysis Based on the Scenario:
- Severity: Not explicitly mentioned but expected to be moderate-to-high due to road closure.
- Location: Highway/Freeway.

### Appropriate Actions:
1. **Deploy IRV** - No (IRV is for vehicle breakdowns or minor issues)
2. **Temporary Lane Closure** - No (full closure is more suitable here)
3. **Use VM

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Received response from OpenAI.
INFO:__main__:Processing question with model gpt-4o.


...

Given the details about the accident (Accident ID A-4732415) and referring to the "Traffic Incident Mitigation Strategies" table, the incident appears to be a significant crash necessitating a road closure with a detour in place in a semi-urban/rural area. It fits best with either Scenario 3 (Major Multi-Vehicle Crash) or Scenario 4 for the high severity and full road closure aspect.

Below is the binary array of actions indicating which actions are relevant (1) or not relevant (0) based on the given table and scenario that best matches the described incident:

- **Deploy IRV**: 1 (Incident Response Vehicle could be deployed)
- **Temporary Lane Closure**: 0 (The road appears to be fully closed, not just a lane)
- **Use VMS to Warn Drivers**: 1 (Variable Message Signs are usually utilized to inform drivers about detours)
- **Notify Police & EMS**: 1 (Given the severity and road closure, notifying emergency services is critical)
- **Quick Clearance Policy**: 0 (The description indicates 

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Received response from OpenAI.
INFO:__main__:Processing question with model gpt-4o.


Based on the information given about the incident and the mitigations outlined in the table, the event can be classified under "Major Multi-Vehicle Crash" with high severity on a highway. The specific actions listed for this scenario are:

- Full or partial lane closures
- Divert traffic to detour routes
- Activate EOC

We would generate a binary array indicating which actions from the actions list are required:

1. Deploy IRV - This was not specified.
2. Temporary Lane Closure - This is included in full or partial lane closures.
3. Use VMS to Warn Drivers - This was not specified explicitly in the major crash scenario but might be implied.
4. Notify Police & EMS - Implicit for major crashes.
5. Quick Clearance Policy - Not specified for high severity.
6. Use VMS & Social Media - Not specifically listed here.
7. Full or Partial Lane Closures - Yes.
8. Divert Traffic to Detour Routes - Yes.
9. Activate EOC - Yes.
10. Full Road Closure - Not specified directly, but full or partial might 

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Received response from OpenAI.


Based on the provided description of the accident record and the information from the "Traffic Incident Mitigation Strategies" table, we can determine the appropriate actions needed to address this incident. The incident is described as a multiple-vehicle crash with roadway closure and northbound traffic being diverted.

Here's the binary array of actions corresponding to the list provided:

1. "Deploy IRV" - 0 (not explicitly mentioned as necessary)
2. "Temporary Lane Closure" - 0 (the whole roadway is closed, so this is not applicable)
3. "Use VMS to Warn Drivers" - 1 (to inform northbound and southbound drivers of the situation)
4. "Notify Police & EMS" - 1 (to manage the crash and provide necessary medical assistance)
5. "Quick Clearance Policy" - 0 (Not mentioned while for major incidents, comprehensive steps are generally taken)
6. "Use VMS & Social Media" - 1 (to keep the public informed about the traffic situation)
7. "Full or Partial Lane Closures" - 1 (full closure of northbo